In [3]:
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import xgboost as xgb
import pandas as pd
import numpy as np
import datetime
import gc
import sys

D:\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


##############################round 2###########################

In [ ]:
summary = train_data.loc[(train_data.is_trade == 1.0) & (train_data.is_trade.isnull() ==False),"context_timestamp_day"].value_counts()
summary = summary.to_frame()
summary.rename(columns={"context_timestamp_day":"y1_sample_count"}, inplace=True)
summary.sort_index(inplace=True)
summary["sample_count"] = train_data.loc[train_data.is_trade.isnull() ==False,"context_timestamp_day"].value_counts()
summary["y1_sample_rate"] = summary["y1_sample_count"] / summary["sample_count"]
user_id_dist = list()
for i in range(8):
    user_id_dist.append(train_data.loc[train_data.context_timestamp_day == i, "user_id"].unique().size)
summary["user_id_dist_count"] = user_id_dist

timestamp_list = list()
time_str_list = ["20180831_120000", "20180901_120000", "20180902_120000", "20180903_120000", 
             "20180904_120000", "20180905_120000", "20180906_120000","20180907_120000"]
for timestr in time_str_list:
    timestamp_list.append(int(datetime.datetime.strptime(timestr,"%Y%m%d_%H%M%S").timestamp())) 
    
# is_trade_am = list()
# is_trade_pm = list()
# click_count_am = list()
# click_count_pm = list()
# for i in range(8):
#     is_trade_am.append(train_data.loc[(train_data.context_timestamp_day == i) & (train_data.context_timestamp < timestamp_list[i]) & (train_data.is_trade == 1.0) , "is_trade"].count())
#     click_count_am.append(train_data.loc[(train_data.context_timestamp_day == i) & (train_data.context_timestamp < timestamp_list[i]), "is_trade"].count())
#     is_trade_pm.append(train_data.loc[(train_data.context_timestamp_day == i) & (train_data.context_timestamp >= timestamp_list[i]) & (train_data.is_trade == 1.0) , "is_trade"].count())
#     click_count_pm.append(train_data.loc[(train_data.context_timestamp_day == i) & (train_data.context_timestamp >= timestamp_list[i]), "is_trade"].count())
    
# summary["y1_sample_count_am"] = is_trade_am
# summary["y1_sample_count_pm"] = is_trade_pm
# summary["sample_count_am"] = click_count_am
# summary["sample_count_pm"] = click_count_pm
# summary["y1_sample_am_rate"] = summary["y1_sample_count_am"] / summary["sample_count_am"]
# summary["y1_sample_pm_rate"] = summary["y1_sample_count_pm"] / summary["sample_count_pm"]

#历史数据定义为2天内
history_dist_user = list()
sample_dist_user = list()
history_sample = list()
is_trade_sample = list()
for i in range(8):
    sample_user_id = train_data.loc[train_data.context_timestamp_day == i,"user_id"].unique()
    history_user_id = train_data.loc[(train_data.context_timestamp_day >= i-2)&(train_data.context_timestamp_day < i),"user_id"].unique()
    sample_history_user_id = np.intersect1d(history_user_id, sample_user_id)
    history_dist_user.append(sample_history_user_id.size)
    sample_dist_user.append(sample_user_id.size)
    
    sample_history_user_sample = pd.DataFrame(sample_history_user_id, columns=["user_id"])
    print(sample_history_user_sample.shape[0])
    sample_history_user_sample = pd.merge(sample_history_user_sample, train_data.loc[(train_data.context_timestamp_day == i)
                                &(train_data.is_trade.isnull()==False),["user_id", "is_trade"]], how="left", on="user_id")
    is_trade_sample.append(sample_history_user_sample.loc[sample_history_user_sample.is_trade == 1.0,"user_id"].shape[0])
    #print(sample_history_user_sample.shape[0])
    history_sample.append(sample_history_user_sample.shape[0])
summary["dist_user_history_count"] = history_dist_user
summary["dist_user_sample_count"] = sample_dist_user
summary["history_sample_count"] = history_sample
summary["y1_history_sample_count"] = is_trade_sample

summary["y1_history_sample_rate"] = summary["y1_history_sample_count"] / summary["history_sample_count"]
summary["dist_user_history_rate"] = summary["dist_user_history_count"] / summary["dist_user_sample_count"]
summary["history_sample_rate"] = summary["history_sample_count"] /summary["sample_count"]

summary["new_sample_count"] =  summary["sample_count"] - summary["history_sample_count"]
summary["y1_new_sample_count"] =  summary["y1_sample_count"] - summary["y1_history_sample_count"]
summary["y1_new_sample_rate"] = summary["y1_new_sample_count"] / summary["new_sample_count"]

0
79554
122537
123494
115351
114697
157074
195001


In [40]:
time_stamp_0700 = int(datetime.datetime.strptime("20180907_000000","%Y%m%d_%H%M%S").timestamp())
time_stamp_0708 = int(datetime.datetime.strptime("20180907_100000","%Y%m%d_%H%M%S").timestamp())
time_stamp_0712 = int(datetime.datetime.strptime("20180907_120000","%Y%m%d_%H%M%S").timestamp())

In [41]:
train_data_7_0008am = train_data.loc[(train_data.context_timestamp >= time_stamp_0700) & (train_data.context_timestamp < time_stamp_0708), :]
train_data_7_0812am = train_data.loc[(train_data.context_timestamp >= time_stamp_0708) & (train_data.context_timestamp < time_stamp_0712), :]

In [682]:
history_dist_user = list()
sample_dist_user = list()
history_sample = list()
is_trade_sample = list()
for i in range(8):
    sample_user_id = train_data.loc[train_data.context_timestamp_day == i,"user_id"].unique()
    history_user_id = train_data.loc[train_data.context_timestamp_day < i,"user_id"].unique()
    sample_history_user_id = np.intersect1d(history_user_id, sample_user_id)
    history_dist_user.append(sample_history_user_id.size)
    sample_dist_user.append(sample_user_id.size)
    
    sample_history_user_sample = pd.DataFrame(sample_history_user_id, columns=["user_id"])
    print(sample_history_user_sample.shape[0])
    sample_history_user_sample = pd.merge(sample_history_user_sample, train_data.loc[(train_data.context_timestamp_day == i)
                                &(train_data.is_trade.isnull()==False),["user_id", "is_trade"]], how="left", on="user_id")
    is_trade_sample.append(sample_history_user_sample.loc[sample_history_user_sample.is_trade == 1.0,"user_id"].shape[0])
    #print(sample_history_user_sample.shape[0])
    history_sample.append(sample_history_user_sample.shape[0])
summary["dist_user_history_count"] = history_dist_user
summary["dist_user_sample_count"] = sample_dist_user
summary["history_sample_count"] = history_sample
summary["y1_history_sample_count"] = is_trade_sample

summary["y1_history_sample_rate"] = summary["y1_history_sample_count"] / summary["history_sample_count"]
summary["dist_user_history_rate"] = summary["dist_user_history_count"] / summary["dist_user_sample_count"]
summary["history_sample_rate"] = summary["history_sample_count"] /summary["sample_count"]

summary["new_sample_count"] =  summary["sample_count"] - summary["history_sample_count"]
summary["y1_new_sample_count"] =  summary["y1_sample_count"] - summary["y1_history_sample_count"]
summary["y1_new_sample_rate"] = summary["y1_new_sample_count"] / summary["new_sample_count"]

0
79554
122537
151973
166167
188817
297221
344204


读取数据 ，将训练数据和测试数据合并

In [4]:
#读取train和test的数据
train_data = pd.read_csv("D:/project/o2o_atom73/atom72_0426/round2_train.txt",sep=" ")
factor_data = ["user_gender_id","user_occupation_id"]
numeric_data =["item_price_level","item_sales_level","item_collected_level",
               "item_pv_level","user_age_level","user_star_level","context_page_id","shop_review_num_level",
               "shop_review_positive_rate","shop_star_level","shop_score_service",
               "shop_score_delivery","shop_score_description"]
#获取广告类型属性的前两个
def get_cp_head_2(x):
    cp_str_list = x.split(";")
    if len(cp_str_list) >=2:
        ret = cp_str_list[0] + ";" + cp_str_list[1]
    else:
        ret = cp_str_list[0]
    return ret
train_data["context_timestamp_hour"] = train_data["context_timestamp"].apply(lambda x: datetime.datetime.fromtimestamp(x).hour)
train_data["context_timestamp_day"] = train_data["context_timestamp"].apply(lambda x:datetime.datetime.fromtimestamp(x).day)
train_data["cp_head_2"] = train_data["predict_category_property"].apply(get_cp_head_2)

train_feature_drop = ["instance_id","is_trade", "shop_id","user_id","item_id","context_timestamp_day","context_timestamp","cp_head_2"]
test_feature_drop = ["instance_id", "shop_id","user_id","item_id","context_timestamp_day","context_timestamp","cp_head_2"]
train_data.drop_duplicates("instance_id", inplace=True)
#31号替换成0，便于特征提取时区分历史特征区间和当前样本区间
train_data.loc[train_data.context_timestamp_day==31,"context_timestamp_day"] = 0

test_data = pd.read_csv("D:/project/o2o_atom73/atom72_0426/round2_test_a.txt",sep=" ")
test_data["context_timestamp_hour"] = test_data["context_timestamp"].apply(lambda x: datetime.datetime.fromtimestamp(x).hour)
test_data["context_timestamp_day"] = test_data["context_timestamp"].apply(lambda x:datetime.datetime.fromtimestamp(x).day)
test_data["cp_head_2"] = test_data["predict_category_property"].apply(get_cp_head_2)

#concat将train 和test行合并时（test缺少label列）会根据列名进行合并，label那一列在test会用NAN填充，
#同时整个数据集的label列的数据类型变成float
train_data = pd.concat([train_data, test_data], axis=0)
#合并a榜和b榜后，需要重新生成索引，否则会出现重复索引 groupby后返回rank信息时会报错
train_data.reset_index(inplace=True,drop=True)
gc.collect()

87

##############################特征提取############################

#训练集 2~6号上午
#验证集 6号下午
#最终没有使用这个方案，直接使用了7号上午作为训练集和验证集
########################for循环提取特征并写入csv文件 #########################

In [4]:
####################################################################1.获取采样周期
timestamp_list = list()
time_str_list = ["20180902_000000", "20180902_120000", "20180903_000000", "20180903_120000", 
             "20180904_000000", "20180904_120000", "20180905_000000", "20180905_120000",
             "20180906_000000", "20180906_120000", "20180907_000000", "20180907_120000",]
for timestr in time_str_list:
    timestamp_list.append(int(datetime.datetime.strptime(timestr,"%Y%m%d_%H%M%S").timestamp())) 

time_2_day = timestamp_list[4] - timestamp_list[0]
time_half_day = timestamp_list[1] - timestamp_list[0]
ts_dataset = pd.DataFrame(data=timestamp_list, columns=["sample_begin"])
ts_dataset["sample_end"] = ts_dataset["sample_begin"] + time_half_day 
ts_dataset["history_begin"] = ts_dataset["sample_begin"] - time_2_day
ts_dataset["half_day_before_sample_begin"] = ts_dataset["sample_begin"] - time_half_day
ts_dataset.reset_index(inplace=True)
####################################################################1.根据采样周期划分样本集
sample_data_dict = dict()
def get_sample_data(x):
    sample_data_dict[x["index"]] = train_data.loc[(train_data["context_timestamp"] >= x["sample_begin"]) & (train_data["context_timestamp"] < x["sample_end"]), :]
ts_dataset.apply(get_sample_data, axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
dtype: object

In [ ]:
################################################### 1 基础特征
base_feature = get_base_feature(train_data, factor_data, numeric_data)
sample_base_feature = pd.concat([train_data[train_feature_drop], base_feature], axis=1, join="inner")
sample_base_feature = sample_base_feature.loc[sample_base_feature["context_timestamp_day"] >=2,:]
sample_base_feature.to_csv("./sample_base_feature.csv", index=False)
gc.collect()
###################################################2.样本特征：广告类别属性
item_cp_feature = get_item_cp_feature(train_data.loc[train_data["context_timestamp_day"] >=2,:])
item_cp_feature.to_csv("./item_cp_feature.csv", index=False)
gc.collect()
#减小内存
train_data.drop("predict_category_property", axis=1, inplace=True)
gc.collect()

###################################################3.历史特征：用户
user_feature_dict = dict()
for i in sample_data_dict:
    print(sample_data_dict[i].shape)
    time_begin = ts_dataset.loc[i, "history_begin"]
    time_end = ts_dataset.loc[i, "sample_begin"]
    user_id = sample_data_dict[i][["user_id"]].drop_duplicates()
    user_history_data = pd.merge(user_id, train_data.loc[(train_data["context_timestamp"] >= time_begin) 
    & (train_data["context_timestamp"] < time_end),:],how="inner", on="user_id")
    user_feature_dict[i] = get_user_feature(sample_data_dict[i],user_history_data)
    del user_history_data
    del user_id
    gc.collect()
user_feature = pd.concat([user_feature_dict[0], user_feature_dict[1],user_feature_dict[2],user_feature_dict[3],
                          user_feature_dict[4],user_feature_dict[5],user_feature_dict[6], user_feature_dict[7],
                          user_feature_dict[8],user_feature_dict[9], user_feature_dict[10],user_feature_dict[11]])
user_feature.to_csv("./user_feature.csv", index=False)
del user_feature_dict
gc.collect()

####################################################历史特征：广告
item_feature_dict = dict()
for i in sample_data_dict:
    print(sample_data_dict[i].shape)
    time_begin = ts_dataset.loc[i, "history_begin"]
    time_end = ts_dataset.loc[i, "sample_begin"]
    item_id = sample_data_dict[i][["item_id"]].drop_duplicates()
    item_history_data = pd.merge(item_id, train_data.loc[(train_data["context_timestamp"] >= time_begin) 
    & (train_data["context_timestamp"] < time_end),:],how="inner", on="item_id")
    item_feature_dict[i] = get_item_feature(sample_data_dict[i],item_history_data)
    del item_history_data
    del item_id
    gc.collect()
item_feature = pd.concat([item_feature_dict[0], item_feature_dict[1],item_feature_dict[2],item_feature_dict[3],
                          item_feature_dict[4],item_feature_dict[5],item_feature_dict[6], item_feature_dict[7],
                          item_feature_dict[8],item_feature_dict[9], item_feature_dict[10],item_feature_dict[11]])
item_feature.to_csv("./item_feature.csv", index=False)
del item_feature_dict
gc.collect()

####################################################历史特征：商店
shop_feature_dict = dict()
for i in sample_data_dict:
    print(sample_data_dict[i].shape)
    time_begin = ts_dataset.loc[i, "history_begin"]
    time_end = ts_dataset.loc[i, "sample_begin"]
    shop_id = sample_data_dict[i][["shop_id"]].drop_duplicates()
    shop_history_data = pd.merge(shop_id, train_data.loc[(train_data["context_timestamp"] >= time_begin) 
    & (train_data["context_timestamp"] < time_end),:],how="inner", on="shop_id")
    shop_feature_dict[i] = get_shop_feature(sample_data_dict[i],shop_history_data)
    del shop_history_data
    del shop_id
    gc.collect()
shop_feature = pd.concat([shop_feature_dict[0],shop_feature_dict[1],shop_feature_dict[2],shop_feature_dict[3],
                         shop_feature_dict[4],shop_feature_dict[5],shop_feature_dict[6],shop_feature_dict[7],
                         shop_feature_dict[8],shop_feature_dict[9],shop_feature_dict[10],shop_feature_dict[11]])
shop_feature.to_csv("./shop_feature.csv", index=False)
del shop_feature_dict
gc.collect()

####################################################广告商店
shopitem_feature_dict = dict()
for i in sample_data_dict:
    print(sample_data_dict[i].shape)
    time_begin = ts_dataset.loc[i, "history_begin"]
    time_end = ts_dataset.loc[i, "sample_begin"]
    shopitem_id = sample_data_dict[i][["shop_id", "item_id"]].drop_duplicates()
    shopitem_history_data = pd.merge(shopitem_id, train_data.loc[(train_data["context_timestamp"] >= time_begin) 
    & (train_data["context_timestamp"] < time_end),:],how="inner", on=["shop_id", "item_id"])
    shopitem_feature_dict[i] = get_shopitem_feature(sample_data_dict[i],shopitem_history_data)
    del shopitem_history_data
    del shopitem_id
    gc.collect()
shopitem_feature = pd.concat([shopitem_feature_dict[0], shopitem_feature_dict[1],shopitem_feature_dict[2],
                              shopitem_feature_dict[3], shopitem_feature_dict[4],shopitem_feature_dict[5],
                              shopitem_feature_dict[6], shopitem_feature_dict[7],shopitem_feature_dict[8],
                              shopitem_feature_dict[9], shopitem_feature_dict[10],shopitem_feature_dict[11]])
shopitem_feature.to_csv("./shopitem_feature.csv", index=False)
del shopitem_feature_dict
gc.collect()

####################################################样本行为特征：用户
user_act_feature_dict = dict()
for i in sample_data_dict:
    print(sample_data_dict[i].shape)
    user_act_feature_dict[i] = get_user_sample_action_feature(sample_data_dict[i])
    gc.collect()
user_sample_action_feature = pd.concat([user_act_feature_dict[0], user_act_feature_dict[1],user_act_feature_dict[2],
                                        user_act_feature_dict[3], user_act_feature_dict[4],user_act_feature_dict[5],
                                        user_act_feature_dict[6], user_act_feature_dict[7],user_act_feature_dict[8],
                                        user_act_feature_dict[9], user_act_feature_dict[10],user_act_feature_dict[11]])
user_sample_action_feature.to_csv("./user_sample_action_feature.csv", index=False)
del user_act_feature_dict
gc.collect()

####################################################样本行为特征V2：用户
user_action_v2_feature_dict = dict()
for i in sample_data_dict:
    print(sample_data_dict[i].shape)
    time_begin = ts_dataset.loc[i, "half_day_before_sample_begin"]
    time_end = ts_dataset.loc[i, "sample_begin"]
    #user_id = sample_data_dict[i][["user_id"]].drop_duplicates()
    user_history_data = train_data.loc[(train_data["context_timestamp"] >= time_begin) 
    & (train_data["context_timestamp"] < time_end),:]
    user_action_v2_feature_dict[i] = get_user_action_v2_feature(sample_data_dict[i], user_history_data)
    del user_history_data
    #del user_id
    gc.collect()
user_action_v2_feature = pd.concat([user_action_v2_feature_dict[0], user_action_v2_feature_dict[1],user_action_v2_feature_dict[2],
                                    user_action_v2_feature_dict[3], user_action_v2_feature_dict[4],user_action_v2_feature_dict[5],
                                    user_action_v2_feature_dict[6], user_action_v2_feature_dict[7],user_action_v2_feature_dict[8],
                                                                                    user_action_v2_feature_dict[9], user_action_v2_feature_dict[10],user_action_v2_feature_dict[11]])
user_action_v2_feature.to_csv("./user_action_v2_feature.csv", index=False)
del user_action_v2_feature_dict
gc.collect()

(368138, 30)
(921338, 30)
(359044, 30)
(877784, 30)
(329331, 30)
(828310, 30)
(328562, 30)
(871657, 30)
(398229, 30)
(1536214, 30)
(1077175, 30)
(519888, 30)
(359044, 30)
(877784, 30)
(329331, 30)
(828310, 30)
(328562, 30)
(871657, 30)
(398229, 30)
(1536214, 30)
(1077175, 30)
(519888, 30)
(368138, 30)
(921338, 30)
(359044, 30)
(877784, 30)
(329331, 30)
(828310, 30)
(328562, 30)
(871657, 30)
(398229, 30)
(1536214, 30)
(1077175, 30)
(519888, 30)
(368138, 30)
(921338, 30)
(359044, 30)
(877784, 30)
(329331, 30)
(828310, 30)
(328562, 30)
(871657, 30)
(398229, 30)
(1536214, 30)
(1077175, 30)
(519888, 30)
(368138, 30)


D:\anaconda\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\sit

(921338, 30)
(359044, 30)
(877784, 30)
(329331, 30)
(828310, 30)
(328562, 30)
(871657, 30)
(398229, 30)
(1536214, 30)
(1077175, 30)
(519888, 30)


0

#训练集 7号0点至8点
#验证集 7号8点~12点

In [54]:
time_stamp_0612 = int(datetime.datetime.strptime("20180906_120000","%Y%m%d_%H%M%S").timestamp())
time_stamp_0700 = int(datetime.datetime.strptime("20180907_000000","%Y%m%d_%H%M%S").timestamp())
train_data_07 = train_data.loc[train_data["context_timestamp_day"]==7, :]

In [56]:
################################################### 1 基础特征
#将基础特征和id信息合并，这里需要确保样本顺序一致，或者直接使用instance_id进行merge
base_feature = get_base_feature(train_data_07, factor_data, numeric_data)
sample_base_feature = pd.concat([train_data_07[train_feature_drop], base_feature], axis=1, join="inner")
sample_base_feature.to_csv("./sample_base_feature.csv", index=False)
gc.collect()
print("base ok")

###################################################2.样本特征：广告类别属性
item_cp_feature = get_item_cp_feature(train_data_07)
item_cp_feature.to_csv("./item_cp_feature.csv", index=False)
gc.collect()
#predict_category_property在后续提取过程不在使用，删除用于减小内存
train_data_07.drop("predict_category_property", axis=1, inplace=True)
gc.collect()
print("cp ok")

###################################################3.历史特征：用户
#历史特征区间为样本前所有天
#优化：根据用户分组，则历史数据仅仅保留样本中存在的用户样本，
#原理是获取历史数据和样本数据中user的交集，使用inner进行merge
user_id = train_data_07[["user_id"]].drop_duplicates()
user_history_data = pd.merge(user_id, train_data.loc[train_data["context_timestamp_day"] <7,:],how="inner", on="user_id")
user_feature = get_user_feature(train_data_07, user_history_data)
user_feature.to_csv("./user_feature.csv", index=False)
gc.collect()
print("user ok")
####################################################历史特征：广告

item_id = train_data_07[["item_id"]].drop_duplicates()
item_history_data = pd.merge(item_id, train_data.loc[train_data["context_timestamp_day"] <7,:] ,how="inner", on="item_id")
item_feature = get_item_feature(train_data_07, item_history_data)
item_feature.to_csv("./item_feature.csv", index=False)
gc.collect()
print("item ok")

####################################################历史特征：商店
shop_id = train_data_07[["shop_id"]].drop_duplicates()
shop_history_data = pd.merge(shop_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on="shop_id")
shop_feature = get_shop_feature(train_data_07, shop_history_data)
shop_feature.to_csv("./shop_feature.csv", index=False)
print("shop ok")

####################################################历史特征:广告品牌
item_brand_id = train_data_07[["user_id", "item_brand_id"]].drop_duplicates()
item_brand_history_data = pd.merge(item_brand_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["user_id", "item_brand_id"])
item_brand_feature = get_item_brand_feature(train_data_07, item_brand_history_data)
item_brand_feature.to_csv("./item_brand_feature.csv", index=False)
gc.collect()
print("item_brand ok")

####################################################历史特征广告商店
shopitem_id = train_data_07[["shop_id", "item_id"]].drop_duplicates()
shopitem_history_data = pd.merge(shopitem_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["shop_id", "item_id"])
shopitem_feature = get_shopitem_feature(train_data_07, shopitem_history_data)
shopitem_feature.to_csv("./shopitem_feature.csv", index=False)
gc.collect()
print("shopitem ok")

####################################################历史特征用户广告
useritem_id = train_data_07[["user_id", "item_id"]].drop_duplicates()
useritem_history_data = pd.merge(useritem_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["user_id", "item_id"])
useritem_feature = get_useritem_feature(train_data_07, useritem_history_data)
useritem_feature.to_csv("./useritem_feature.csv", index=False)
gc.collect()
print("useritem ok")

####################################################历史特征用户商店
usershop_id = train_data_07[["user_id", "shop_id"]].drop_duplicates()
usershop_history_data = pd.merge(usershop_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["user_id", "shop_id"])
usershop_feature = get_usershop_feature(train_data_07, usershop_history_data)
usershop_feature.to_csv("./usershop_feature.csv", index=False)
gc.collect()
print("usershop ok")

####################################################历史特征用户广告品牌
useritem_brand_id = train_data_07[["user_id", "item_brand_id"]].drop_duplicates()
useritem_brand_history_data = pd.merge(useritem_brand_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["user_id", "item_brand_id"])
useritem_brand_feature = get_useritem_brand_feature(train_data_07, useritem_brand_history_data)
useritem_brand_feature.to_csv("./useritem_brand_feature.csv", index=False)
gc.collect()
print("useritem_brand ok")

####################################################历史样本差分特征：广告属性差异
item_id = train_data_07[["item_id"]].drop_duplicates()
item_history_data = pd.merge(item_id, train_data.loc[train_data["context_timestamp_day"] <7,:] ,how="inner", on="item_id")
item_diff_feature = get_item_diff_feture(train_data_07, item_history_data)
item_diff_feature.to_csv("./item_diff_feature.csv", index=False)
gc.collect()
print("item_diff_feature ok")
####################################################样本行为特征：用户
user_sample_action_feature = get_user_sample_action_feature(train_data_07)
user_sample_action_feature.to_csv("./user_sample_action_feature.csv", index=False)
gc.collect()
print("user action ok")

####################################################样本行为特征V2：用户 上一次，下一次的点击时间间隔
#用户行为特征包括了样本前半天的用户数据，主要用于提取时间信息
#不仅仅是对用户分组，还有对用户广告，用户商店分组，因此没有在函数外对历史用户进行merge
user_history_data = train_data.loc[(train_data["context_timestamp"] >= time_stamp_0612) 
& (train_data["context_timestamp"] < time_stamp_0700),:]
user_action_v2_feature = get_user_action_v2_feature(train_data_07, user_history_data)
user_action_v2_feature.to_csv("./user_action_v2_feature.csv", index=False)
gc.collect()
print("user action v2 ok")
####################################################样本行为特征V3：用户在上/下一个小时内的点击数
#用户行为特征包括了样本前一小时的用户数据，主要用于提取时间信息
#不仅仅是对用户分组，还有对用户广告，用户商店分组，因此没有在函数外对历史用户进行merge
user_history_data = train_data.loc[(train_data["context_timestamp"] >= (time_stamp_0700-3600)) 
& (train_data["context_timestamp"] < time_stamp_0700),:]
user_action_v3_feature = get_user_action_v3_feature(train_data_07, user_history_data)
user_action_v3_feature.to_csv("./user_action_v3_feature.csv", index=False)
gc.collect()
print("user action v3 ok")

base ok


D:\anaconda\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\sit

cp ok
user ok
item ok
shop ok
shopitem ok


D:\anaconda\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\sit

user action ok


D:\anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
D:\anaconda\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
D:\anaconda\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anac

user action v2 ok


#############################特征提取接口###############################

In [43]:
#######################################################样本基础特征
def get_base_feature(input_data, factor_data, numeric_data):   
    base_data = input_data.copy()
    factor_feature_data = pd.get_dummies(base_data[factor_data], columns=factor_data)
    numeric_feature_data = base_data[numeric_data]
    base_feature = pd.concat([factor_feature_data, numeric_feature_data], axis=1)
    return base_feature

In [44]:
#######################################################样本广告类别属性和预测类别属性特征
def get_pred_category_list(x):
    category_list = list()
    for categeory in x:
        category_list.append(categeory.split(":")[0])
    return category_list
#获取真实类目在预测类目列表中的索引排序
def get_pred_category_index(x):
    #print(x["category"])
    #print(x["pred_cageg"])
    if x["item_category"] in x["pred_category_list"]:
        #索引/数量 索引不为0 因此将索引+1 同时融入了没有找到该索引的信息
        return x["pred_category_list"].index(x["item_category"]) + 1
    else:
        return 0

def get_true_category_property_in_pred_list(x):
    if x["category_index_in_pred_category_list"] == 0:
        return "NULL" #返回"NULL"
    else:
        return x["pred_category_property_list"][x["category_index_in_pred_category_list"] - 1]

def get_all_property_list(x):
    #print(x)
    property_list = list()
    for property_str in x:
        #print(string.split(":")[0])
        property_list.append(property_str.split(":")[-1])
    #print(ret_list)
    return property_list

def get_pred_category_which_has_pro_list(x):
    #print(x)
    category_list = list()
    for category in x:
        #print(string.split(":")[0])
        category_pro = category.split(":")
        if category_pro[-1] != "-1":
            category_list.append(category_pro[0])
    return category_list

def get_pred_category_which_has_pro_index(x):
    #print(x["category"])
    #print(x["pred_cageg"])
    if x["item_category"] in x["pred_category_which_has_pro_list"]:
        #索引/数量 索引不为0 因此将索引+1 同时融入了没有找到该索引的信息
        return x["pred_category_which_has_pro_list"].index(x["item_category"]) + 1
    else:
        return 0
#category_index_in_pred_category_list: 当前样本真实类目在预测得到的类目列表中的排序索引
##category_index_div_pred_num: pred_category_num/category_index_in_pred_category_list
##pred_pro_intersection_item_pro_num：当前样本的上下文预测得到的广告类目与真实广告类目相符的那条信息的属性个数和真实属性的交集
##pred_pro_intersection_item_pro_num_div_pred_property_num  pred_pro_intersection_item_pro_num/pred_property_num
##pred_pro_intersection_item_pro_num_div_item_property_num  pred_pro_intersection_item_pro_num/item_property_num
##pred_category_which_property_is_-1_num:当前样本的所有预测属性中-1的个数
##pred_category_which_property_is_-1_num_div_pred_category_num：当前样本中预测的属性中-1的个数与所有属性个数之比
#pred_category_which_has_pro_num：当前样本中预测的广告类型中拥有属性的个数
#pred_category_which_has_pro_num_div_pred_category_num:当前样本中预测的广告类型中拥有属性的个数与总类型数之比
def get_item_cp_feature(input_data):
    category_data = input_data
    category_feature = category_data[["instance_id","item_category_list", "predict_category_property","item_property_list"]]
    #item_category:当前样本所属的真实广告类目
    #pred_category_property_list:当前样本上下文预测得到的类目属性列表
    #pred_category_list:当前样本上下文预测得到的类目列表
    #pred_category_num: 当前样本上下文预测得到的类目个数
    category_feature["item_category"] = category_feature["item_category_list"].apply(lambda x: x.split(";")[-1])
    category_feature["pred_category_property_list"] = category_feature["predict_category_property"].apply(lambda x: x.split(";"))
    category_feature["pred_category_list"] = category_feature["pred_category_property_list"].apply(get_pred_category_list)
    category_feature["category_index_in_pred_category_list"] = category_feature.apply(get_pred_category_index, axis=1)
    category_feature["pred_category_num"] = category_feature.apply(lambda x: len(x["pred_category_list"]), axis=1)
    category_feature["category_index_div_pred_num"] = category_feature["category_index_in_pred_category_list"] / category_feature["pred_category_num"]
    gc.collect()

    #item_property_list: 当前样本的广告在某个类目下的真实属性列表
    #item_property_num: 当前样本的广告在某个类目下的真实属性个数
    #true_category_property_in_pred_list 当前样本的上下文预测得到的广告类目和真实广告类目相符的那条信息
    #pred_property_list：当前样本的上下文预测得到的广告类目与真实广告类目相符的那条信息的属性列表
    #pred_property_num：当前样本的上下文预测得到的广告类目与真实广告类目相符的那条信息的属性个数
    
    category_feature["item_property_list"] = category_feature.item_property_list.apply(lambda x:x.split(";"))
    #真实广告属于同一个类别，但会拥有不同的属性
    category_feature["item_property_num"] = category_feature.item_property_list.apply(lambda x:len(x))    
    #上下文预测中和广告类别预测一致的信息
    category_feature["true_category_property_in_pred_list"] = category_feature.apply(get_true_category_property_in_pred_list, axis=1)
    category_feature["pred_property_list"] = category_feature["true_category_property_in_pred_list"].apply(lambda x:x.split(":")[-1].split(","))
    category_feature["pred_property_num"] = category_feature["pred_property_list"].apply(lambda x:len(x))
    #set的结果是元组
    category_feature["pred_pro_intersection_item_pro_num"] = category_feature.apply(lambda x: len(set(x["pred_property_list"]).intersection(set(x["item_property_list"]))), axis=1)
    gc.collect()

    #pred_pro_intersection_item_pro_num_div_pred_property_num  pred_pro_intersection_item_pro_num/pred_property_num
    #pred_pro_intersection_item_pro_num_div_item_property_num  pred_pro_intersection_item_pro_num/item_property_num
    #pred_all_property_list 
    #pred_category_which_property_is_
    #pred_category_which_property_is_
    category_feature["pred_pro_intersection_item_pro_num_div_pred_property_num"] = category_feature.apply(lambda x:x["pred_pro_intersection_item_pro_num"]/x["pred_property_num"],axis=1)
    category_feature["pred_pro_intersection_item_pro_num_div_item_property_num"] = category_feature.apply(lambda x:x["pred_pro_intersection_item_pro_num"]/x["item_property_num"],axis=1)
    category_feature["pred_all_property_list"] = category_feature.pred_category_property_list.apply(get_all_property_list)
    category_feature["pred_category_which_property_is_-1_num"] = category_feature["pred_all_property_list"].apply(lambda x:x.count('-1'))
    category_feature["pred_category_which_property_is_-1_num_div_pred_category_num"] = category_feature.apply(lambda x:x["pred_category_which_property_is_-1_num"]/x["pred_category_num"], axis=1)
    gc.collect()
    category_feature["pred_category_which_has_pro_list"] =  category_feature.pred_category_property_list.apply(get_pred_category_which_has_pro_list)
    category_feature["pred_category_which_has_pro_num"] = category_feature.apply(get_pred_category_which_has_pro_index, axis=1)
    category_feature["pred_category_which_has_pro_num_div_pred_category_num"] = category_feature.apply(lambda x: x["pred_category_which_has_pro_num"]/x["pred_category_num"], axis=1)
    gc.collect()
    train_category_feature = category_feature[["instanpred_category_which_property_is_ce_id","category_index_in_pred_category_list","category_index_div_pred_num","pred_pro_intersection_item_pro_num",
        "pred_pro_intersection_item_pro_num_div_pred_property_num","pred_pro_intersection_item_pro_num_div_item_property_num",
                    "pred_category_which_property_is_-1_num","pred_category_which_property_is_-1_num_div_pred_category_num",
                    "pred_category_which_has_pro_num","pred_category_which_has_pro_num_div_pred_category_num"]]
    return train_category_feature
################################广告类目属性特征######################################

In [45]:
######################################################历史转化率：商店评分变化
def get_shop_diff(input_data, history_day):
    
    shop_data = input_data.copy()
    date_index = shop_data.context_timestamp_day.unique()
    date_index.sort()
    shop_data_current_day = shop_data.loc[shop_data.context_timestamp_day == date_index[-1],:]
    shop_data_history_day = shop_data.loc[(shop_data.context_timestamp_day < date_index[-1]) & (shop_data.context_timestamp_day >= date_index[-1-history_day]),:]
    #shop_data = shop_data_history_day
    
    shop_data = shop_data.loc[(shop_data.context_timestamp_day <= date_index[-1]) & (shop_data.context_timestamp_day >= date_index[-1-history_day]),:]
    shop_data = shop_data.drop_duplicates(["shop_id", "context_timestamp_day"])
    shop_id = shop_data[["shop_id"]].drop_duplicates()
    date_index = shop_data.context_timestamp_day.unique()
    date_index.sort()

    t1 = shop_data[["shop_id", "context_timestamp_day", "shop_review_num_level"]]
    t1 = t1.set_index(["shop_id", "context_timestamp_day"])
    t1 = t1.unstack()
    t1 = t1.reset_index()
    t1["shop_shop_review_num_level_diff"] = t1.shop_review_num_level[date_index[2]] *7000 - t1.shop_review_num_level[date_index[1]]*4000 -t1.shop_review_num_level[date_index[0]]*3000
    t1["shop_shop_review_num_level_diff"] = t1["shop_shop_review_num_level_diff"]/t1.shop_review_num_level[date_index[0]]
    t1 = t1[["shop_shop_review_num_level_diff", "shop_id"]]
    t1.columns = t1.columns.droplevel(1)

    t2 = shop_data[["shop_id", "context_timestamp_day", "shop_review_positive_rate"]]
    t2 = t2.set_index(["shop_id", "context_timestamp_day"])
    t2 = t2.unstack()
    t2 = t2.reset_index()
    t2["shop_shop_review_positive_rate_diff"] = t2.shop_review_positive_rate[date_index[2]] *7000 - t2.shop_review_positive_rate[date_index[1]]*4000 -t2.shop_review_positive_rate[date_index[0]]*3000
    t2["shop_shop_review_positive_rate_diff"] = t2["shop_shop_review_positive_rate_diff"]/t2.shop_review_positive_rate[date_index[0]]
    t2 = t2[["shop_shop_review_positive_rate_diff", "shop_id"]]
    t2.columns = t2.columns.droplevel(1)

    t3 = shop_data[["shop_id", "context_timestamp_day", "shop_star_level"]]
    t3 = t3.set_index(["shop_id", "context_timestamp_day"])
    t3 = t3.unstack()
    t3 = t3.reset_index()
    t3["shop_shop_star_level_diff"] = t3.shop_star_level[date_index[2]] *7000 - t3.shop_star_level[date_index[1]]*4000 -t3.shop_star_level[date_index[0]]*3000
    t3["shop_shop_star_level_diff"] = t3["shop_shop_star_level_diff"]/t3.shop_star_level[date_index[0]]
    t3 = t3[["shop_shop_star_level_diff", "shop_id"]]
    t3.columns = t3.columns.droplevel(1)

    t4 = shop_data[["shop_id", "context_timestamp_day", "shop_score_service"]]
    t4 = t4.set_index(["shop_id", "context_timestamp_day"])
    t4 = t4.unstack()
    t4 = t4.reset_index()
    t4["shop_shop_score_service_diff"] = t4.shop_score_service[date_index[2]] *7000 - t4.shop_score_service[date_index[1]]*4000 -t4.shop_score_service[date_index[0]]*3000
    t4["shop_shop_score_service_diff"] = t4["shop_shop_score_service_diff"]/t4.shop_score_service[date_index[0]]
    t4 = t4[["shop_shop_score_service_diff", "shop_id"]]
    t4.columns = t4.columns.droplevel(1)

    t5 = shop_data[["shop_id", "context_timestamp_day", "shop_score_delivery"]]
    t5 = t5.set_index(["shop_id", "context_timestamp_day"])
    t5 = t5.unstack()
    t5 = t5.reset_index()
    t5["shop_shop_score_delivery_diff"] = t5.shop_score_delivery[date_index[2]] *7000 - t5.shop_score_delivery[date_index[1]]*4000 -t5.shop_score_delivery[date_index[0]]*3000
    t5["shop_shop_score_delivery_diff"] = t5["shop_shop_score_delivery_diff"]/t5.shop_score_delivery[date_index[0]]
    t5 = t5[["shop_shop_score_delivery_diff", "shop_id"]]
    t5.columns = t5.columns.droplevel(1)


    t6 = shop_data[["shop_id", "context_timestamp_day", "shop_score_description"]]
    t6 = t6.set_index(["shop_id", "context_timestamp_day"])
    t6 = t6.unstack()
    t6 = t6.reset_index()

    t6["shop_shop_score_description_diff"] = t6.shop_score_description[date_index[2]] *7000 - t6.shop_score_description[date_index[1]]*4000 -t6.shop_score_description[date_index[0]]*3000
    t6["shop_shop_score_description_diff"] = t6["shop_shop_score_description_diff"]/t6.shop_score_description[date_index[0]]
    t6 = t6[["shop_shop_score_description_diff", "shop_id"]]
    t6.columns = t6.columns.droplevel(1)

    shop_diff_feature = pd.merge(shop_id, t1, how="left", on="shop_id")
    shop_diff_feature = pd.merge(shop_diff_feature, t2, how="left", on="shop_id")
    shop_diff_feature = pd.merge(shop_diff_feature, t3, how="left", on="shop_id")
    shop_diff_feature = pd.merge(shop_diff_feature, t4, how="left", on="shop_id")
    shop_diff_feature = pd.merge(shop_diff_feature, t5, how="left", on="shop_id")
    shop_diff_feature = pd.merge(shop_diff_feature, t6, how="left", on="shop_id")
    shop_diff_feature.fillna(value=0, inplace=True)
    
    shop_sample_feture = shop_data_current_day[["instance_id", "shop_id"]]
    shop_diff_feature = pd.merge(shop_sample_feture, shop_diff_feature, how="left", on="shop_id")
    shop_diff_feature.drop("shop_id", inplace=True, axis=1)
    return shop_diff_feature

In [ ]:
shop_item_dist_count                          #商店中的不同广告个数
shop_y1_item_dist_count                       #商店中购买的不同广告个数
shop_y1_item_count_div_item_count             #商店中广告的购买率
shop_y1_item_count_div_item_dist_count        #商店中购买的广告个数比不同的广告个数
shop_y1_item_dist_count_div_item_dist_coun    #商店中购买的不同广告个数比不同的广告个数

In [46]:
######################################################历史转化率：商店
def get_shop_feature(sample_data, history_data):
    shop_data_current_day = sample_data
    shop_data = history_data

    shop_id = shop_data.loc[:,["shop_id"]].drop_duplicates()
    shop_data_is_trade = shop_data.loc[shop_data["is_trade"] == 1, :]
    shop_group = shop_data.groupby("shop_id")
    shop_y1_group = shop_data_is_trade.groupby("shop_id")

    tf0 = shop_group[["item_id"]].count()
    tf0.rename(columns={"item_id":"shop_count"},inplace=True)
    #广告统计
    tf1_1 = tf0.copy()
    tf1_1["shop_item_dist_count"] = shop_group["item_id"].agg(lambda x:x.unique().size)
    tf1_2 = shop_y1_group[["item_id"]].count()
    tf1_2.rename(columns={"item_id": "shop_y1_item_count"},inplace=True)
    tf1_2["shop_y1_item_dist_count"] = shop_y1_group["item_id"].agg(lambda x:x.unique().size)
    tf1_1.reset_index(inplace=True)
    tf1_2.reset_index(inplace=True)
    tf1_tmp = pd.merge(left=tf1_1, right=tf1_2, how="left", on="shop_id")
    tf1_tmp["shop_y1_item_count_div_item_count"] = tf1_tmp["shop_y1_item_count"] / tf1_tmp["shop_count"]
    tf1_tmp["shop_y1_item_count_div_item_dist_count"] = tf1_tmp["shop_y1_item_count"] / tf1_tmp["shop_item_dist_count"]
    tf1_tmp["shop_y1_item_dist_count_div_item_dist_count"] = tf1_tmp["shop_y1_item_dist_count"] / tf1_tmp["shop_item_dist_count"]
    tf1_tmp.fillna(value=0, inplace=True)
    tf1 = tf1_tmp.drop(labels="shop_count", axis=1)
    #用户统计
    tf2_1 = tf0.copy()
    tf2_1["shop_user_dist_count"] = shop_group["user_id"].agg(lambda x:x.unique().size)
    tf2_2 = shop_y1_group[["user_id"]].count()
    tf2_2.rename(columns={"user_id": "shop_y1_user_count"},inplace=True)
    tf2_2["shop_y1_user_dist_count"] = shop_y1_group["user_id"].agg(lambda x:x.unique().size)
    tf2_1.reset_index(inplace=True)
    tf2_2.reset_index(inplace=True)
    tf2_tmp = pd.merge(left=tf2_1, right=tf2_2, how="left", on="shop_id")
    tf2_tmp["shop_y1_user_count_div_user_count"] = tf2_tmp["shop_y1_user_count"] / tf2_tmp["shop_count"]
    tf2_tmp["shop_y1_user_count_div_user_dist_count"] = tf2_tmp["shop_y1_user_count"] / tf2_tmp["shop_user_dist_count"]
    tf2_tmp["shop_y1_user_dist_count_div_user_dist_count"] = tf2_tmp["shop_y1_user_dist_count"] / tf2_tmp["shop_user_dist_count"]
    tf2_tmp.fillna(value=0, inplace=True)
    tf2 = tf2_tmp.drop(labels="shop_count", axis=1)
    del tf2_1, tf2_2
    gc.collect()
    #广告品牌统计
    tf3_1 = tf0.copy()
    tf3_1["shop_item_brand_dist_count"] = shop_group["item_brand_id"].agg(lambda x:x[x!=-1].unique().size)
    tf3_2 = shop_y1_group[["item_brand_id"]].count()
    tf3_2.rename(columns={"item_brand_id": "shop_y1_item_brand_count"},inplace=True)
    tf3_2["shop_y1_item_brand_dist_count"] = shop_y1_group["item_brand_id"].agg(lambda x:x[x!=-1].unique().size)
    tf3_1.reset_index(inplace=True)
    tf3_2.reset_index(inplace=True)
    tf3_tmp = pd.merge(left=tf3_1, right=tf3_2, how="left", on="shop_id")
    tf3_tmp["shop_y1_item_brand_count_div_item_brand_count"] = tf3_tmp["shop_y1_item_brand_count"] / tf3_tmp["shop_count"]
    tf3_tmp["shop_y1_item_brand_count_div_item_brand_dist_count"] = tf3_tmp["shop_y1_item_brand_count"] / tf3_tmp["shop_item_brand_dist_count"]
    tf3_tmp["shop_y1_item_brand_dist_count_div_item_brand_dist_count"] = tf3_tmp["shop_y1_item_brand_dist_count"] / tf3_tmp["shop_item_brand_dist_count"]
    tf1_tmp.fillna(value=0, inplace=True)
    tf3 = tf3_tmp.drop(labels="shop_count", axis=1)
    del tf3_1, tf3_2
    gc.collect()
    #城市统计
    tf4_1 = tf0.copy()
    tf4_group = shop_data.loc[shop_data["item_city_id"] != -1,:].groupby("shop_id")
    tf4_1["shop_item_city_dist_count"] = tf4_group["item_city_id"].agg(lambda x:x[x!=-1].unique().size)
    tf4_2 = shop_y1_group[["item_city_id"]].count()
    tf4_2.rename(columns={"item_city_id": "shop_y1_item_city_count"},inplace=True)
    tf4_2["shop_y1_item_city_dist_count"] = shop_y1_group["item_city_id"].agg(lambda x:x[x!=-1].unique().size)
    tf4_1.reset_index(inplace=True)
    tf4_2.reset_index(inplace=True)
    tf4_tmp = pd.merge(left=tf4_1, right=tf4_2, how="left", on="shop_id")
    tf4_tmp["shop_y1_item_city_count_div_item_city_count"] = tf4_tmp["shop_y1_item_city_count"] / tf4_tmp["shop_count"]
    tf4_tmp["shop_y1_item_city_count_div_item_city_dist_count"] = tf4_tmp["shop_y1_item_city_count"] / tf4_tmp["shop_item_city_dist_count"]
    tf4_tmp["shop_y1_item_city_dist_count_div_item_city_dist_count"] = tf4_tmp["shop_y1_item_city_dist_count"] / tf4_tmp["shop_item_city_dist_count"]
    tf4_tmp.fillna(value=0, inplace=True)
    tf4 = tf4_tmp.drop(labels="shop_count", axis=1)
    #tf4 = tf4_tmp.loc[:,["shop_id", "shop_y1_item_city_count_div_item_city_count", "shop_y1_item_city_count_div_item_city_dist_count", "shop_y1_item_city_dist_count_div_item_city_dist_count"]]
    del tf4_1, tf4_2
    gc.collect()
    #搜索属性统计
    tf5_1 = tf0.copy()
    tf5_group = shop_data.loc[shop_data["cp_head_2"] != -1,:].groupby("shop_id")
    tf5_1["shop_predict_cp_dist_count"] = tf5_group["cp_head_2"].agg(lambda x:x[x!=-1].unique().size)
    tf5_2 = shop_y1_group[["cp_head_2"]].count()
    tf5_2.rename(columns={"cp_head_2": "shop_y1_predict_cp_count"},inplace=True)
    tf5_2["shop_y1_predict_cp_dist_count"] = shop_y1_group["cp_head_2"].agg(lambda x:x[x!=-1].unique().size)
    tf5_1.reset_index(inplace=True)
    tf5_2.reset_index(inplace=True)
    tf5_tmp = pd.merge(left=tf5_1, right=tf5_2, how="left", on="shop_id")
    tf5_tmp["shop_y1_predict_cp_count_div_predict_cp_count"] = tf5_tmp["shop_y1_predict_cp_count"] / tf5_tmp["shop_count"]
    tf5_tmp["shop_y1_predict_cp_count_div_predict_cp_dist_count"] = tf5_tmp["shop_y1_predict_cp_count"] / tf5_tmp["shop_predict_cp_dist_count"]
    tf5_tmp["shop_y1_predict_cp_dist_count_div_predict_cp_dist_count"] = tf5_tmp["shop_y1_predict_cp_dist_count"] / tf5_tmp["shop_predict_cp_dist_count"]
    tf5_tmp.fillna(value=0, inplace=True)
    tf5 = tf5_tmp.drop(labels="shop_count", axis=1)
    del tf5_1, tf5_2
    gc.collect()
    #店铺中交易成功的广告商品的价格等级,销量等级，被收藏次数的等级，展示次数的等级的平均值
    
    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["item_sales_level"] !=-1, ["shop_id", "item_id", "item_sales_level"]].drop_duplicates()
    tn1_2_group = tmp_data.groupby("shop_id")
    tn1_2 = tn1_2_group[["item_sales_level"]].mean()
    tn1_2.rename(columns={"item_sales_level":"shop_y1_item_sales_level_mean"}, inplace=True)
    tn1_2.reset_index(inplace=True)
    tn1_tmp = pd.merge(left=shop_id, right=tn1_2, how="left", on="shop_id")
    tn1 = tn1_tmp

    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["item_price_level"] !=-1, ["shop_id", "item_id", "item_price_level"]].drop_duplicates()
    tn2_2_group = tmp_data.groupby("shop_id")
    tn2_2 = tn2_2_group[["item_price_level"]].mean()
    tn2_2.rename(columns={"item_price_level":"shop_y1_item_price_level_mean"}, inplace=True)
    tn2_2.reset_index(inplace=True)
    tn2 = tn2_tmp

    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["item_collected_level"] !=-1, ["shop_id", "item_id", "item_collected_level"]].drop_duplicates()
    tn3_2_group = tmp_data.groupby("shop_id")
    tn3_2 = tn3_2_group[["item_collected_level"]].mean()
    tn3_2.rename(columns={"item_collected_level":"shop_y1_item_collected_level_mean"}, inplace=True)
    tn3_2.reset_index(inplace=True)
    tn3_tmp = pd.merge(left=shop_id, right=tn3_2, how="left", on="shop_id")
    tn3 = tn3_tmp
    
    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["item_pv_level"] !=-1, ["shop_id", "item_id", "item_pv_level"]].drop_duplicates()
    tn4_2_group = tmp_data.groupby("shop_id")
    tn4_2 = tn4_2_group[["item_pv_level"]].mean()
    tn4_2.rename(columns={"item_pv_level":"shop_y1_item_pv_level_mean"}, inplace=True)
    tn4_2.reset_index(inplace=True)
    tn4_tmp = pd.merge(left=shop_id, right=tn4_2, how="left", on="shop_id")
    tn4 = tn4_tmp

    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["user_age_level"] !=-1,:]
    tn5_2_group = tmp_data.groupby("shop_id")
    tn5_2 = tn5_2_group[["user_age_level"]].mean()
    tn5_2.rename(columns={"user_age_level":"shop_y1_user_age_level_mean"}, inplace=True)
    tn5_2.reset_index(inplace=True)
    tn5_tmp = pd.merge(left=shop_id, right=tn5_2, how="left", on="shop_id")
    tn5 = tn5_tmp

    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["user_star_level"] !=-1,:]
    tn6_2_group = tmp_data.groupby("shop_id")
    tn6_2 = tn6_2_group[["user_star_level"]].mean()
    tn6_2.rename(columns={"user_star_level":"shop_y1_user_star_level_mean"}, inplace=True)
    tn6_2.reset_index(inplace=True)
    tn6_tmp = pd.merge(left=shop_id, right=tn6_2, how="left", on="shop_id")
    tn6 = tn6_tmp
    
    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["context_page_id"] !=-1,:]
    tn7_2_group = tmp_data.groupby("shop_id")
    tn7_2 = tn7_2_group[["context_page_id"]].mean()
    tn7_2.rename(columns={"context_page_id":"shop_y1_context_page_id_mean"}, inplace=True)
    tn7_2.reset_index(inplace=True)
    tn7_tmp = pd.merge(left=shop_id, right=tn7_2, how="left", on="shop_id")
    tn7 = tn7_tmp
    
    tmp_data = shop_data.loc[shop_data["shop_review_num_level"] !=-1, :]
    tn8_1 = tmp_data.groupby("shop_id")[["shop_review_num_level"]].mean().reset_index()
    tn8_1.rename(columns={"shop_review_num_level":"shop_shop_review_num_level_mean"}, inplace=True)
    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["shop_review_num_level"] !=-1,:]
    tn8_2_group = tmp_data.groupby("shop_id")
    tn8_2 = tn8_2_group[["shop_review_num_level"]].mean()
    tn8_2.rename(columns={"shop_review_num_level":"shop_y1_shop_review_num_level_mean"}, inplace=True)
    tn8_2.reset_index(inplace=True)
    tn8_tmp = pd.merge(left=shop_id, right=tn8_1, how="left", on="shop_id")
    tn8_tmp = pd.merge(left=tn8_tmp, right=tn8_2, how="left", on="shop_id")
    tn8 = tn8_tmp

    tmp_data = shop_data.loc[shop_data["shop_review_positive_rate"] !=-1, :]
    tn9_1 = tmp_data.groupby("shop_id")[["shop_review_positive_rate"]].mean().reset_index()
    tn9_1.rename(columns={"shop_review_positive_rate":"shop_shop_review_positive_rate_mean"}, inplace=True)
    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["shop_review_positive_rate"] !=-1,:]
    tn9_2_group = tmp_data.groupby("shop_id")
    tn9_2 = tn9_2_group[["shop_review_positive_rate"]].mean()
    tn9_2.rename(columns={"shop_review_positive_rate":"shop_y1_shop_review_positive_rate_mean"}, inplace=True)
    tn9_2.reset_index(inplace=True)
    tn9_tmp = pd.merge(left=shop_id, right=tn9_1, how="left", on="shop_id")
    tn9_tmp = pd.merge(left=tn9_tmp, right=tn9_2, how="left", on="shop_id")
    tn9 = tn9_tmp

    tmp_data = shop_data.loc[shop_data["shop_star_level"] !=-1, :]
    tn10_1 = tmp_data.groupby("shop_id")[["shop_star_level"]].mean().reset_index()
    tn10_1.rename(columns={"shop_star_level":"shop_shop_star_level_mean"}, inplace=True)
    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["shop_star_level"] !=-1,:]
    tn10_2_group = tmp_data.groupby("shop_id")
    tn10_2 = tn10_2_group[["shop_star_level"]].mean()
    tn10_2.rename(columns={"shop_star_level":"shop_y1_shop_star_level_mean"}, inplace=True)
    tn10_2.reset_index(inplace=True)
    tn10_tmp = pd.merge(left=shop_id, right=tn10_1, how="left", on="shop_id")
    tn10_tmp = pd.merge(left=tn10_tmp, right=tn10_2, how="left", on="shop_id")
    tn10 = tn10_tmp

    tmp_data = shop_data.loc[shop_data["shop_score_delivery"] !=-1, :]
    tn11_1 = tmp_data.groupby("shop_id")[["shop_score_delivery"]].mean().reset_index()
    tn11_1.rename(columns={"shop_score_delivery":"shop_shop_score_delivery_mean"}, inplace=True)
    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["shop_score_delivery"] !=-1,:]
    tn11_2_group = tmp_data.groupby("shop_id")
    tn11_2 = tn11_2_group[["shop_score_delivery"]].mean()
    tn11_2.rename(columns={"shop_score_delivery":"shop_y1_shop_score_delivery_mean"}, inplace=True)
    tn11_2.reset_index(inplace=True)
    tn11_tmp = pd.merge(left=shop_id, right=tn11_1, how="left", on="shop_id")
    tn11_tmp = pd.merge(left=tn11_tmp, right=tn11_2, how="left", on="shop_id")
    tn11 = tn11_tmp

    tmp_data = shop_data.loc[shop_data["shop_score_description"] !=-1, :]
    tn12_1 = tmp_data.groupby("shop_id")[["shop_score_description"]].mean().reset_index()
    tn12_1.rename(columns={"shop_score_description":"shop_shop_score_description_mean"}, inplace=True)
    tmp_data = shop_data_is_trade.loc[shop_data_is_trade["shop_score_description"] !=-1,:]
    tn12_2_group = tmp_data.groupby("shop_id")
    tn12_2 = tn12_2_group[["shop_score_description"]].mean()
    tn12_2.rename(columns={"shop_score_description":"shop_y1_shop_score_description_mean"}, inplace=True)
    tn12_2.reset_index(inplace=True)
    tn12_tmp = pd.merge(left=shop_id, right=tn12_1, how="left", on="shop_id")
    tn12_tmp = pd.merge(left=tn12_tmp, right=tn12_2, how="left", on="shop_id")
    tn12 = tn12_tmp
    
    shop_id = shop_data.loc[:,["shop_id"]].drop_duplicates()

    shop_feature = tf0.reset_index()
    shop_feature = pd.merge(left=shop_feature, right=tf1, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tf2, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tf3, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tf4, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tf5, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn1, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn2, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn3, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn4, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn5, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn6, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn7, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn8, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn9, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn10, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn11, how="left", on="shop_id")
    shop_feature = pd.merge(left=shop_feature, right=tn12, how="left", on="shop_id")
    #shop_feature = pd.merge(left=shop_feature, right=to1, how="left", on="shop_id")
    
    shop_sample_feture = shop_data_current_day[["instance_id", "shop_id"]]
    shop_feature = pd.merge(shop_sample_feture, shop_feature, how="left", on="shop_id")
    shop_feature.drop("shop_id", inplace=True, axis=1)
    return shop_feature

In [47]:
######################################################历史转化率：广告
def get_item_feature(sample_data, history_data):
    item_data_current_day = sample_data
    item_data = history_data
    
    item_id = item_data.loc[:,["item_id"]].drop_duplicates()
    item_data_is_trade = item_data.loc[item_data["is_trade"] == 1, :]
    item_group = item_data.groupby("item_id")
    item_y1_group = item_data_is_trade.groupby("item_id")

    tf0 = item_group[["user_id"]].count()
    tf0.rename(columns={"user_id":"item_count"},inplace=True)
    #用户统计
    tf1_1 = tf0.copy()
    tf1_1["item_user_dist_count"] = item_group["user_id"].agg(lambda x:x.unique().size)
    tf1_2 = item_y1_group[["user_id"]].count()
    tf1_2.rename(columns={"user_id": "item_y1_user_count"},inplace=True)
    tf1_2["item_y1_user_dist_count"] = item_y1_group["user_id"].agg(lambda x:x.unique().size)
    tf1_1.reset_index(inplace=True)
    tf1_2.reset_index(inplace=True)
    tf1_tmp = pd.merge(left=tf1_1, right=tf1_2, how="left", on="item_id")
    tf1_tmp["item_y1_user_count_div_user_count"] = tf1_tmp["item_y1_user_count"] / tf1_tmp["item_count"]
    tf1_tmp["item_y1_user_count_div_user_dist_count"] = tf1_tmp["item_y1_user_count"] / tf1_tmp["item_user_dist_count"]
    tf1_tmp["item_y1_user_dist_count_div_user_dist_count"] = tf1_tmp["item_y1_user_dist_count"] / tf1_tmp["item_user_dist_count"]
    tf1_tmp.fillna(value=0, inplace=True)
    tf1 = tf1_tmp.drop(labels="item_count", axis=1)

    #商店统计
    tf2_1 = tf0.copy()
    tf2_1["item_shop_dist_count"] = item_group["shop_id"].agg(lambda x:x.unique().size)
    tf2_2 = item_y1_group[["shop_id"]].count()
    tf2_2.rename(columns={"shop_id": "item_y1_shop_count"},inplace=True)
    tf2_2["item_y1_shop_dist_count"] = item_y1_group["shop_id"].agg(lambda x:x.unique().size)
    tf2_1.reset_index(inplace=True)
    tf2_2.reset_index(inplace=True)
    tf2_tmp = pd.merge(left=tf2_1, right=tf2_2, how="left", on="item_id")
    tf2_tmp["item_y1_shop_count_div_shop_count"] = tf2_tmp["item_y1_shop_count"] / tf2_tmp["item_count"]
    tf2_tmp["item_y1_shop_count_div_shop_dist_count"] = tf2_tmp["item_y1_shop_count"] / tf2_tmp["item_shop_dist_count"]
    tf2_tmp["item_y1_shop_dist_count_div_shop_dist_count"] = tf2_tmp["item_y1_shop_dist_count"] / tf2_tmp["item_shop_dist_count"]
    tf2_tmp.fillna(value=0, inplace=True)
    tf2 = tf2_tmp.drop(labels="item_count", axis=1)

    #店铺中交易成功的广告商品的价格等级,销量等级，被收藏次数的等级，展示次数的等级的平均值
    tmp_data = item_data_is_trade.loc[item_data_is_trade["user_age_level"] !=-1,:]
    tn5_2_group = tmp_data.groupby("item_id")
    tn5_2 = tn5_2_group[["user_age_level"]].mean()
    tn5_2.rename(columns={"user_age_level":"item_y1_user_age_level_mean"}, inplace=True)
    tn5_2.reset_index(inplace=True)
    tn5_tmp = pd.merge(left=item_id, right=tn5_2, how="left", on="item_id")
    tn5 = tn5_tmp

    tmp_data = item_data_is_trade.loc[item_data_is_trade["user_star_level"] !=-1,:]
    tn6_2_group = tmp_data.groupby("item_id")
    tn6_2 = tn6_2_group[["user_star_level"]].mean()
    tn6_2.rename(columns={"user_star_level":"item_y1_user_star_level_mean"}, inplace=True)
    tn6_2.reset_index(inplace=True)
    tn6_tmp = pd.merge(left=item_id, right=tn6_2, how="left", on="item_id")
    tn6 = tn6_tmp

    tmp_data = item_data_is_trade.loc[item_data_is_trade["shop_review_num_level"] !=-1,:]
    tn8_2_group = tmp_data.groupby("item_id")
    tn8_2 = tn8_2_group[["shop_review_num_level"]].mean()
    tn8_2.rename(columns={"shop_review_num_level":"item_y1_shop_review_num_level_mean"}, inplace=True)
    tn8_2.reset_index(inplace=True)
    tn8_tmp = pd.merge(left=item_id, right=tn8_2, how="left", on="item_id")
    tn8 = tn8_tmp

    tmp_data = item_data_is_trade.loc[item_data_is_trade["shop_review_positive_rate"] !=-1,:]
    tn9_2_group = tmp_data.groupby("item_id")
    tn9_2 = tn9_2_group[["shop_review_positive_rate"]].mean()
    tn9_2.rename(columns={"shop_review_positive_rate":"item_y1_shop_review_positive_rate_mean"}, inplace=True)
    tn9_2.reset_index(inplace=True)
    tn9_tmp = pd.merge(left=item_id, right=tn9_2, how="left", on="item_id")
    tn9 = tn9_tmp

    tmp_data = item_data_is_trade.loc[item_data_is_trade["shop_star_level"] !=-1,:]
    tn10_2_group = tmp_data.groupby("item_id")
    tn10_2 = tn10_2_group[["shop_star_level"]].mean()
    tn10_2.rename(columns={"shop_star_level":"item_y1_shop_star_level_mean"}, inplace=True)
    tn10_2.reset_index(inplace=True)
    tn10_tmp = pd.merge(left=item_id, right=tn10_2, how="left", on="item_id")
    tn10 = tn10_tmp

    tmp_data = item_data_is_trade.loc[item_data_is_trade["shop_score_delivery"] !=-1,:]
    tn11_2_group = tmp_data.groupby("item_id")
    tn11_2 = tn11_2_group[["shop_score_delivery"]].mean()
    tn11_2.rename(columns={"shop_score_delivery":"item_y1_shop_score_delivery_mean"}, inplace=True)
    tn11_2.reset_index(inplace=True)
    tn11_tmp = pd.merge(left=item_id, right=tn11_2, how="left", on="item_id")
    tn11 = tn11_tmp

    tmp_data = item_data_is_trade.loc[item_data_is_trade["shop_score_description"] !=-1,:]
    tn12_2_group = tmp_data.groupby("item_id")
    tn12_2 = tn12_2_group[["shop_score_description"]].mean()
    tn12_2.rename(columns={"shop_score_description":"item_y1_shop_score_description_mean"}, inplace=True)
    tn12_2.reset_index(inplace=True)
    tn12_tmp = pd.merge(left=item_id, right=tn12_2, how="left", on="item_id")
    tn12 = tn12_tmp

    item_feature = tf0.reset_index()
    item_feature = pd.merge(left=item_feature, right=tf1, how="left", on="item_id")
    item_feature = pd.merge(left=item_feature, right=tf2, how="left", on="item_id")
    item_feature = pd.merge(left=item_feature, right=tn5, how="left", on="item_id")
    item_feature = pd.merge(left=item_feature, right=tn8, how="left", on="item_id")
    item_feature = pd.merge(left=item_feature, right=tn9, how="left", on="item_id")
    item_feature = pd.merge(left=item_feature, right=tn10, how="left", on="item_id")
    item_feature = pd.merge(left=item_feature, right=tn11, how="left", on="item_id")
    item_feature = pd.merge(left=item_feature, right=tn12, how="left", on="item_id")
    #item_feature = pd.merge(left=item_feature, right=to1, how="left", on="item_id")
    
    item_sample_feture = item_data_current_day[["instance_id", "item_id"]]
    item_feature = pd.merge(item_sample_feture, item_feature, how="left", on="item_id")
    item_feature.drop("item_id", inplace=True, axis=1)
    return item_feature

In [ ]:
######################################################历史转化率：广告品牌ID
def get_item_brand_feature(sample_data, history_data):
    #item_brand_data_current_day=train_data_07
    #item_brand_data = item_brand_history_data
    item_brand_data_current_day = sample_data
    item_brand_data = history_data

    item_brand_id = item_brand_data.loc[:,["item_brand_id"]].drop_duplicates()
    item_brand_data_is_trade = item_brand_data.loc[item_brand_data["is_trade"] == 1, :]
    item_brand_group = item_brand_data.groupby("item_brand_id")
    item_brand_y1_group = item_brand_data_is_trade.groupby("item_brand_id")

    tf0 = item_brand_group[["instance_id"]].count()
    tf0.rename(columns={"instance_id":"item_brand_count"},inplace=True)

    tf1_1 = tf0.copy()
    tf1_1["item_brand_item_dist_count"] = item_brand_group["item_id"].agg(lambda x:x.unique().size)
    tf1_2 = item_brand_y1_group[["item_id"]].count()
    tf1_2.rename(columns={"item_id": "item_brand_y1_item_count"},inplace=True)
    tf1_2["item_brand_y1_item_dist_count"] = item_brand_y1_group["item_id"].agg(lambda x:x.unique().size)
    tf1_1.reset_index(inplace=True)
    tf1_2.reset_index(inplace=True)
    tf1_tmp = pd.merge(left=tf1_1, right=tf1_2, how="left", on="item_brand_id")
    tf1_tmp["item_brand_y1_item_count_div_item_count"] = tf1_tmp["item_brand_y1_item_count"] / tf1_tmp["item_brand_count"]
    tf1_tmp["item_brand_y1_item_count_div_item_dist_count"] = tf1_tmp["item_brand_y1_item_count"] / tf1_tmp["item_brand_item_dist_count"]
    tf1_tmp["item_brand_y1_item_dist_count_div_item_dist_count"] = tf1_tmp["item_brand_y1_item_dist_count"] / tf1_tmp["item_brand_item_dist_count"]
    tf1_tmp.fillna(value=0, inplace=True)
    tf1 = tf1_tmp.drop(labels="item_brand_count", axis=1)
    del tf1_1, tf1_2
    gc.collect()
    tf2_1 = tf0.copy()
    tf2_1["item_brand_user_dist_count"] = item_brand_group["user_id"].agg(lambda x:x.unique().size)
    tf2_2 = item_brand_y1_group[["user_id"]].count()
    tf2_2.rename(columns={"user_id": "item_brand_y1_user_count"},inplace=True)
    tf2_2["item_brand_y1_user_dist_count"] = item_brand_y1_group["user_id"].agg(lambda x:x.unique().size)
    tf2_1.reset_index(inplace=True)
    tf2_2.reset_index(inplace=True)
    tf2_tmp = pd.merge(left=tf2_1, right=tf2_2, how="left", on="item_brand_id")
    tf2_tmp["item_brand_y1_user_count_div_user_count"] = tf2_tmp["item_brand_y1_user_count"] / tf2_tmp["item_brand_count"]
    tf2_tmp["item_brand_y1_user_count_div_user_dist_count"] = tf2_tmp["item_brand_y1_user_count"] / tf2_tmp["item_brand_user_dist_count"]
    tf2_tmp["item_brand_y1_user_dist_count_div_user_dist_count"] = tf2_tmp["item_brand_y1_user_dist_count"] / tf2_tmp["item_brand_user_dist_count"]
    tf2_tmp.fillna(value=0, inplace=True)
    tf2 = tf2_tmp.drop(labels="item_brand_count", axis=1)
    #tf2 = tf2_tmp.loc[:,["item_brand_id", "item_brand_y1_user_count_div_user_count", "item_brand_y1_user_count_div_user_dist_count", "item_brand_y1_user_dist_count_div_user_dist_count"]]
    del tf2_1, tf2_2
    gc.collect()

    tf3_1 = tf0.copy()
    tf3_1["item_brand_shop_dist_count"] = item_brand_group["shop_id"].agg(lambda x:x[x!=-1].unique().size)
    tf3_2 = item_brand_y1_group[["shop_id"]].count()
    tf3_2.rename(columns={"shop_id": "item_brand_y1_shop_count"},inplace=True)
    tf3_2["item_brand_y1_shop_dist_count"] = item_brand_y1_group["shop_id"].agg(lambda x:x[x!=-1].unique().size)
    tf3_1.reset_index(inplace=True)
    tf3_2.reset_index(inplace=True)
    tf3_tmp = pd.merge(left=tf3_1, right=tf3_2, how="left", on="item_brand_id")
    tf3_tmp["item_brand_y1_shop_count_div_shop_count"] = tf3_tmp["item_brand_y1_shop_count"] / tf3_tmp["item_brand_count"]
    tf3_tmp["item_brand_y1_shop_count_div_shop_dist_count"] = tf3_tmp["item_brand_y1_shop_count"] / tf3_tmp["item_brand_shop_dist_count"]
    tf3_tmp["item_brand_y1_shop_dist_count_div_shop_dist_count"] = tf3_tmp["item_brand_y1_shop_dist_count"] / tf3_tmp["item_brand_shop_dist_count"]
    tf1_tmp.fillna(value=0, inplace=True)
    tf3 = tf3_tmp.drop(labels="item_brand_count", axis=1)
    del tf3_1, tf3_2
    gc.collect()

    tf4_1 = tf0.copy()
    tf4_group = item_brand_data.loc[item_brand_data["item_city_id"] != -1,:].groupby("item_brand_id")
    tf4_1["item_brand_item_city_dist_count"] = tf4_group["item_city_id"].agg(lambda x:x[x!=-1].unique().size)
    tf4_2 = item_brand_y1_group[["item_city_id"]].count()
    tf4_2.rename(columns={"item_city_id": "item_brand_y1_item_city_count"},inplace=True)
    tf4_2["item_brand_y1_item_city_dist_count"] = item_brand_y1_group["item_city_id"].agg(lambda x:x[x!=-1].unique().size)
    tf4_1.reset_index(inplace=True)
    tf4_2.reset_index(inplace=True)
    tf4_tmp = pd.merge(left=tf4_1, right=tf4_2, how="left", on="item_brand_id")
    tf4_tmp["item_brand_y1_item_city_count_div_item_city_count"] = tf4_tmp["item_brand_y1_item_city_count"] / tf4_tmp["item_brand_count"]
    tf4_tmp["item_brand_y1_item_city_count_div_item_city_dist_count"] = tf4_tmp["item_brand_y1_item_city_count"] / tf4_tmp["item_brand_item_city_dist_count"]
    tf4_tmp["item_brand_y1_item_city_dist_count_div_item_city_dist_count"] = tf4_tmp["item_brand_y1_item_city_dist_count"] / tf4_tmp["item_brand_item_city_dist_count"]
    tf4_tmp.fillna(value=0, inplace=True)
    tf4 = tf4_tmp.drop(labels="item_brand_count", axis=1)
    del tf4_1, tf4_2
    gc.collect()

    tf5_1 = tf0.copy()
    tf5_group = item_brand_data.loc[item_brand_data["cp_head_2"] != -1,:].groupby("item_brand_id")
    tf5_1["item_brand_predict_cp_dist_count"] = tf5_group["cp_head_2"].agg(lambda x:x[x!=-1].unique().size)
    tf5_2 = item_brand_y1_group[["cp_head_2"]].count()
    tf5_2.rename(columns={"cp_head_2": "item_brand_y1_predict_cp_count"},inplace=True)
    tf5_2["item_brand_y1_predict_cp_dist_count"] = item_brand_y1_group["cp_head_2"].agg(lambda x:x[x!=-1].unique().size)
    tf5_1.reset_index(inplace=True)
    tf5_2.reset_index(inplace=True)
    tf5_tmp = pd.merge(left=tf5_1, right=tf5_2, how="left", on="item_brand_id")
    tf5_tmp["item_brand_y1_predict_cp_count_div_predict_cp_count"] = tf5_tmp["item_brand_y1_predict_cp_count"] / tf5_tmp["item_brand_count"]
    tf5_tmp["item_brand_y1_predict_cp_count_div_predict_cp_dist_count"] = tf5_tmp["item_brand_y1_predict_cp_count"] / tf5_tmp["item_brand_predict_cp_dist_count"]
    tf5_tmp["item_brand_y1_predict_cp_dist_count_div_predict_cp_dist_count"] = tf5_tmp["item_brand_y1_predict_cp_dist_count"] / tf5_tmp["item_brand_predict_cp_dist_count"]
    tf5_tmp.fillna(value=0, inplace=True)
    tf5 = tf5_tmp.drop(labels="item_brand_count", axis=1)
    del tf5_1, tf5_2
    gc.collect()
    #店铺中交易成功的广告商品的价格等级,销量等级，被收藏次数的等级，展示次数的等级的平均值
    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["item_sales_level"] !=-1, ["item_brand_id", "item_id", "item_sales_level"]].drop_duplicates()
    tn1_2_group = tmp_data.groupby("item_brand_id")
    tn1_2 = tn1_2_group[["item_sales_level"]].mean()
    tn1_2.rename(columns={"item_sales_level":"item_brand_y1_item_sales_level_mean"}, inplace=True)
    tn1_2.reset_index(inplace=True)
    tn1_tmp = pd.merge(left=item_brand_id, right=tn1_2, how="left", on="item_brand_id")
    tn1 = tn1_tmp

    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["item_price_level"] !=-1, ["item_brand_id", "item_id", "item_price_level"]].drop_duplicates()
    tn2_2_group = tmp_data.groupby("item_brand_id")
    tn2_2 = tn2_2_group[["item_price_level"]].mean()
    tn2_2.rename(columns={"item_price_level":"item_brand_y1_item_price_level_mean"}, inplace=True)
    tn2_2.reset_index(inplace=True)
    tn2_tmp = pd.merge(left=item_brand_id, right=tn2_2, how="left", on="item_brand_id")
    tn2 = tn2_tmp

    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["item_collected_level"] !=-1, ["item_brand_id", "item_id", "item_collected_level"]].drop_duplicates()
    tn3_2_group = tmp_data.groupby("item_brand_id")
    tn3_2 = tn3_2_group[["item_collected_level"]].mean()
    tn3_2.rename(columns={"item_collected_level":"item_brand_y1_item_collected_level_mean"}, inplace=True)
    tn3_2.reset_index(inplace=True)
    tn3_tmp = pd.merge(left=item_brand_id, right=tn3_2, how="left", on="item_brand_id")
    tn3 = tn3_tmp

    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["item_pv_level"] !=-1, ["item_brand_id", "item_id", "item_pv_level"]].drop_duplicates()
    tn4_2_group = tmp_data.groupby("item_brand_id")
    tn4_2 = tn4_2_group[["item_pv_level"]].mean()
    tn4_2.rename(columns={"item_pv_level":"item_brand_y1_item_pv_level_mean"}, inplace=True)
    tn4_2.reset_index(inplace=True)
    tn4_tmp = pd.merge(left=item_brand_id, right=tn4_2, how="left", on="item_brand_id")
    tn4 = tn4_tmp

    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["user_age_level"] !=-1,:]
    tn5_2_group = tmp_data.groupby("item_brand_id")
    tn5_2 = tn5_2_group[["user_age_level"]].mean()
    tn5_2.rename(columns={"user_age_level":"item_brand_y1_user_age_level_mean"}, inplace=True)
    tn5_2.reset_index(inplace=True)
    tn5_tmp = pd.merge(left=item_brand_id, right=tn5_2, how="left", on="item_brand_id")
    tn5 = tn5_tmp

    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["user_star_level"] !=-1,:]
    tn6_2_group = tmp_data.groupby("item_brand_id")
    tn6_2 = tn6_2_group[["user_star_level"]].mean()
    tn6_2.rename(columns={"user_star_level":"item_brand_y1_user_star_level_mean"}, inplace=True)
    tn6_2.reset_index(inplace=True)
    tn6_tmp = pd.merge(left=item_brand_id, right=tn6_2, how="left", on="item_brand_id")
    tn6 = tn6_tmp

    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["context_page_id"] !=-1,:]
    tn7_2_group = tmp_data.groupby("item_brand_id")
    tn7_2 = tn7_2_group[["context_page_id"]].mean()
    tn7_2.rename(columns={"context_page_id":"item_brand_y1_context_page_id_mean"}, inplace=True)
    tn7_2.reset_index(inplace=True)
    tn7_tmp = pd.merge(left=item_brand_id, right=tn7_2, how="left", on="item_brand_id")
     tn7 = tn7_tmp

    tmp_data = item_brand_data.loc[item_brand_data["shop_review_num_level"] !=-1, :]
    tn8_1 = tmp_data.groupby("item_brand_id")[["shop_review_num_level"]].mean().reset_index()
    tn8_1.rename(columns={"shop_review_num_level":"item_brand_shop_review_num_level_mean"}, inplace=True)
    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["shop_review_num_level"] !=-1,:]
    tn8_2_group = tmp_data.groupby("item_brand_id")
    tn8_2 = tn8_2_group[["shop_review_num_level"]].mean()
    tn8_2.rename(columns={"shop_review_num_level":"item_brand_y1_shop_review_num_level_mean"}, inplace=True)
    tn8_2.reset_index(inplace=True)
    tn8_tmp = pd.merge(left=item_brand_id, right=tn8_1, how="left", on="item_brand_id")
    tn8_tmp = pd.merge(left=tn8_tmp, right=tn8_2, how="left", on="item_brand_id")
    tn8 = tn8_tmp

    tmp_data = item_brand_data.loc[item_brand_data["shop_review_positive_rate"] !=-1, :]
    tn9_1 = tmp_data.groupby("item_brand_id")[["shop_review_positive_rate"]].mean().reset_index()
    tn9_1.rename(columns={"shop_review_positive_rate":"item_brand_shop_review_positive_rate_mean"}, inplace=True)
    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["shop_review_positive_rate"] !=-1,:]
    tn9_2_group = tmp_data.groupby("item_brand_id")
    tn9_2 = tn9_2_group[["shop_review_positive_rate"]].mean()
    tn9_2.rename(columns={"shop_review_positive_rate":"item_brand_y1_shop_review_positive_rate_mean"}, inplace=True)
    tn9_2.reset_index(inplace=True)
    tn9_tmp = pd.merge(left=item_brand_id, right=tn9_1, how="left", on="item_brand_id")
    tn9_tmp = pd.merge(left=tn9_tmp, right=tn9_2, how="left", on="item_brand_id")
    tn9 = tn9_tmp

    tmp_data = item_brand_data.loc[item_brand_data["shop_star_level"] !=-1, :]
    tn10_1 = tmp_data.groupby("item_brand_id")[["shop_star_level"]].mean().reset_index()
    tn10_1.rename(columns={"shop_star_level":"item_brand_shop_star_level_mean"}, inplace=True)
    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["shop_star_level"] !=-1,:]
    tn10_2_group = tmp_data.groupby("item_brand_id")
    tn10_2 = tn10_2_group[["shop_star_level"]].mean()
    tn10_2.rename(columns={"shop_star_level":"item_brand_y1_shop_star_level_mean"}, inplace=True)
    tn10_2.reset_index(inplace=True)
    tn10_tmp = pd.merge(left=item_brand_id, right=tn10_1, how="left", on="item_brand_id")
    tn10_tmp = pd.merge(left=tn10_tmp, right=tn10_2, how="left", on="item_brand_id")
    tn10 = tn10_tmp

    tmp_data = item_brand_data.loc[item_brand_data["shop_score_delivery"] !=-1, :]
    tn11_1 = tmp_data.groupby("item_brand_id")[["shop_score_delivery"]].mean().reset_index()
    tn11_1.rename(columns={"shop_score_delivery":"item_brand_shop_score_delivery_mean"}, inplace=True)
    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["shop_score_delivery"] !=-1,:]
    tn11_2_group = tmp_data.groupby("item_brand_id")
    tn11_2 = tn11_2_group[["shop_score_delivery"]].mean()
    tn11_2.rename(columns={"shop_score_delivery":"item_brand_y1_shop_score_delivery_mean"}, inplace=True)
    tn11_2.reset_index(inplace=True)
    tn11_tmp = pd.merge(left=item_brand_id, right=tn11_1, how="left", on="item_brand_id")
    tn11_tmp = pd.merge(left=tn11_tmp, right=tn11_2, how="left", on="item_brand_id")
    tn11 = tn11_tmp

    tmp_data = item_brand_data.loc[item_brand_data["shop_score_description"] !=-1, :]
    tn12_1 = tmp_data.groupby("item_brand_id")[["shop_score_description"]].mean().reset_index()
    tn12_1.rename(columns={"shop_score_description":"item_brand_shop_score_description_mean"}, inplace=True)
    tmp_data = item_brand_data_is_trade.loc[item_brand_data_is_trade["shop_score_description"] !=-1,:]
    tn12_2_group = tmp_data.groupby("item_brand_id")
    tn12_2 = tn12_2_group[["shop_score_description"]].mean()
    tn12_2.rename(columns={"shop_score_description":"item_brand_y1_shop_score_description_mean"}, inplace=True)
    tn12_2.reset_index(inplace=True)
    tn12_tmp = pd.merge(left=item_brand_id, right=tn12_1, how="left", on="item_brand_id")
    tn12_tmp = pd.merge(left=tn12_tmp, right=tn12_2, how="left", on="item_brand_id")
    tn12 = tn12_tmp

    item_brand_id = item_brand_data.loc[:,["item_brand_id"]].drop_duplicates()

    item_brand_feature = tf0.reset_index()
    item_brand_feature = pd.merge(left=item_brand_feature, right=tf1, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tf2, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tf3, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tf4, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tf5, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn1, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn2, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn3, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn4, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn5, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn6, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn7, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn8, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn9, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn10, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn11, how="left", on="item_brand_id")
    item_brand_feature = pd.merge(left=item_brand_feature, right=tn12, how="left", on="item_brand_id")

    item_brand_sample_feture = item_brand_data_current_day[["instance_id", "item_brand_id"]]
    item_brand_feature = pd.merge(item_brand_sample_feture, item_brand_feature, how="left", on="item_brand_id")
    item_brand_feature.drop("item_brand_id", inplace=True, axis=1)
    return item_brand_feature

In [ ]:
######################################################历史转化率：用户
def get_user_feature(sample_data, history_data):
    user_data_current_day = sample_data
    user_data_history_day = history_data
    user_data_y1_history_day = user_data_history_day.loc[user_data_history_day.is_trade == 1,:]
    user_id_current_day = user_data_current_day.loc[:,["user_id"]].drop_duplicates()
###################################用户历史点击次数，历史点击天数
    th_1 = user_data_history_day.groupby("user_id")[["instance_id"]].count()
    th_1["user_day_count"] = user_data_history_day.groupby("user_id")["context_timestamp_day"].agg(lambda x:x.unique().size)
    th_1.reset_index(inplace=True)
    th_1.rename(columns={"instance_id":"user_count"},inplace=True)
    
    th_2 = user_data_y1_history_day.groupby("user_id")[["instance_id"]].count().reset_index()
    th_2.rename(columns={"instance_id":"user_y1_count"},inplace=True)
    t_tmp = pd.merge(th_1, th_2, how="left", on="user_id")
    t_tmp["user_y1_count_div_user_count"] = t_tmp["user_y1_count"] / t_tmp["user_count"]
    user_id_history_feature = pd.merge(user_id_current_day, t_tmp, how="left", on="user_id")
    user_id_history_feature.fillna(0, inplace=True)
    
#     t1 = user_data_current_day.groupby("user_id")[["item_id"]].count()
#     t1.rename(columns={"item_id":"user_count_today"},inplace=True)
#     t1["user_item_dist_count_today"] = user_data_current_day.groupby("user_id")["item_id"].agg(lambda x:x.unique().size)
#     t1["user_shop_dist_count_doday"] = user_data_current_day.groupby("user_id")["shop_id"].agg(lambda x:x.unique().size)
#     t1.reset_index(inplace=True)
#     t1.fillna(0, inplace=True)
#     user_feature_tmp = pd.merge(user_id_history_feature, t1, how="left", on="user_id")

#     user_data_current_day["user_click_rank_today"] = user_data_current_day.loc[:, ["user_id", "context_timestamp"]].groupby("user_id")["context_timestamp"].rank(ascending=1,method='min')

    user_sample_feture = user_data_current_day[["instance_id", "user_id"]]
    user_feature = pd.merge(user_sample_feture, user_id_history_feature, how="left", on="user_id")
    user_feature.drop("user_id", inplace=True, axis=1)
    return user_feature

In [48]:
######################################################历史转化率：商店与广告
def get_shopitem_feature(sample_data, history_data):
    shopitem_data_current_day = sample_data
    shopitem_data = history_data 
    
    shopitem_id = shopitem_data.loc[:,["shop_id", "item_id"]].drop_duplicates()
    shopitem_data_is_trade = shopitem_data.loc[shopitem_data["is_trade"] == 1, :]
    shopitem_group = shopitem_data.groupby(["shop_id", "item_id"])
    shopitem_y1_group = shopitem_data_is_trade.groupby(["shop_id", "item_id"])

    tf0 = shopitem_group[["user_id"]].count()
    tf0.rename(columns={"user_id":"shopitem_count"},inplace=True)
    #用户统计
    tf1_1 = tf0.copy()
    tf1_1["shopitem_user_dist_count"] = shopitem_group["user_id"].agg(lambda x:x.unique().size)
    tf1_2 = shopitem_y1_group[["user_id"]].count()
    tf1_2.rename(columns={"user_id": "shopitem_y1_user_count"},inplace=True)
    tf1_2["shopitem_y1_user_dist_count"] = shopitem_y1_group["user_id"].agg(lambda x:x.unique().size)
    tf1_1.reset_index(inplace=True)
    tf1_2.reset_index(inplace=True)
    tf1_tmp = pd.merge(left=tf1_1, right=tf1_2, how="left", on=["shop_id", "item_id"])
    tf1_tmp["shopitem_y1_user_count_div_user_count"] = tf1_tmp["shopitem_y1_user_count"] / tf1_tmp["shopitem_count"]
    tf1_tmp["shopitem_y1_user_count_div_user_dist_count"] = tf1_tmp["shopitem_y1_user_count"] / tf1_tmp["shopitem_user_dist_count"]
    tf1_tmp["shopitem_y1_user_dist_count_div_user_dist_count"] = tf1_tmp["shopitem_y1_user_dist_count"] / tf1_tmp["shopitem_user_dist_count"]
    tf1_tmp.fillna(value=0, inplace=True)
    tf1 = tf1_tmp
    shopitem_feature = pd.merge(left=shopitem_id, right=tf1, how="left", on=["shop_id", "item_id"])
    
    shopitem_sample_feture = shopitem_data_current_day[["instance_id", "shop_id", "item_id"]]
    shopitem_feature = pd.merge(shopitem_sample_feture, shopitem_feature, how="left", on=["shop_id","item_id"])
    shopitem_feature.drop(["shop_id","item_id"], inplace=True, axis=1)

    return shopitem_feature

In [ ]:
######################################################历史转化率：用户与广告
def get_useritem_feature(sample_data, history_data):

    #useritem_id = train_data_07[["user_id", "item_id"]].drop_duplicates()
    #useritem_history_data = pd.merge(useritem_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["user_id", "item_id"])
    #useritem_data = pd.merge(useritem_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["user_id", "item_id"])
    useritem_data_current_day = sample_data
    useritem_data = history_data 

    useritem_id = useritem_data.loc[:,["user_id", "item_id"]].drop_duplicates()
    useritem_data_is_trade = useritem_data.loc[useritem_data["is_trade"] == 1, :]
    useritem_group = useritem_data.groupby(["user_id", "item_id"])
    useritem_y1_group = useritem_data_is_trade.groupby(["user_id", "item_id"])

    tf0 = useritem_group[["instance_id"]].count()
    tf0.rename(columns={"instance_id":"useritem_count"},inplace=True)

    tf1_1 = tf0.copy()
    tf1_1["useritem_shop_dist_count"] = useritem_group["shop_id"].agg(lambda x:x.unique().size)
    tf1_2 = useritem_y1_group[["instance_id"]].count()
    tf1_2.rename(columns={"instance_id": "useritem_y1_shop_count"},inplace=True)
    tf1_2["useritem_y1_shop_dist_count"] = useritem_y1_group["user_id"].agg(lambda x:x.unique().size)
    tf1_1.reset_index(inplace=True)
    tf1_2.reset_index(inplace=True)
    tf1_tmp = pd.merge(left=tf1_1, right=tf1_2, how="left", on=["user_id", "item_id"])
    tf1_tmp["useritem_y1_shop_count_div_shop_count"] = tf1_tmp["useritem_y1_shop_count"] / tf1_tmp["useritem_count"]
    tf1_tmp["useritem_y1_shop_count_div_shop_dist_count"] = tf1_tmp["useritem_y1_shop_count"] / tf1_tmp["useritem_shop_dist_count"]
    tf1_tmp["useritem_y1_shop_dist_count_div_shop_dist_count"] = tf1_tmp["useritem_y1_shop_dist_count"] / tf1_tmp["useritem_shop_dist_count"]
    tf1_tmp.fillna(value=0, inplace=True)
    tf1 = tf1_tmp
    #tf1 = tf1_tmp.drop(labels="useritem_count", axis=1)
    useritem_feature = pd.merge(left=useritem_id, right=tf1, how="left", on=["user_id", "item_id"])

    useritem_sample_feture = useritem_data_current_day[["instance_id", "user_id", "item_id"]]
    useritem_feature = pd.merge(useritem_sample_feture, useritem_feature, how="left", on=["user_id","item_id"])
    useritem_feature.drop(["user_id","item_id"], inplace=True, axis=1)
    useritem_feature.fillna(value=0, inplace=True)
    return useritem_feature

In [ ]:
######################################################历史转化率：用户与商店
def get_usershop_feature(sample_data, history_data):

    #usershop_id = train_data_07[["user_id", "shop_id"]].drop_duplicates()
    #usershop_history_data = pd.merge(usershop_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["user_id", "shop_id"])
    #usershop_data = pd.merge(usershop_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["user_id", "shop_id"])
    usershop_data_current_day = sample_data
    usershop_data = history_data 

    usershop_id = usershop_data.loc[:,["user_id", "shop_id"]].drop_duplicates()
    usershop_data_is_trade = usershop_data.loc[usershop_data["is_trade"] == 1, :]
    usershop_group = usershop_data.groupby(["user_id", "shop_id"])
    usershop_y1_group = usershop_data_is_trade.groupby(["user_id", "shop_id"])

    tf0 = usershop_group[["instance_id"]].count()
    tf0.rename(columns={"instance_id":"usershop_count"},inplace=True)

    tf1_1 = tf0.copy()
    tf1_1["usershop_item_dist_count"] = usershop_group["item_id"].agg(lambda x:x.unique().size)
    tf1_2 = usershop_y1_group[["instance_id"]].count()
    tf1_2.rename(columns={"instance_id": "usershop_y1_item_count"},inplace=True)
    tf1_2["usershop_y1_item_dist_count"] = usershop_y1_group["user_id"].agg(lambda x:x.unique().size)
    tf1_1.reset_index(inplace=True)
    tf1_2.reset_index(inplace=True)
    tf1_tmp = pd.merge(left=tf1_1, right=tf1_2, how="left", on=["user_id", "shop_id"])
    tf1_tmp["usershop_y1_item_count_div_item_count"] = tf1_tmp["usershop_y1_item_count"] / tf1_tmp["usershop_count"]
    tf1_tmp["usershop_y1_item_count_div_item_dist_count"] = tf1_tmp["usershop_y1_item_count"] / tf1_tmp["usershop_item_dist_count"]
    tf1_tmp["usershop_y1_item_dist_count_div_item_dist_count"] = tf1_tmp["usershop_y1_item_dist_count"] / tf1_tmp["usershop_item_dist_count"]
    tf1_tmp.fillna(value=0, inplace=True)
    tf1 = tf1_tmp
    #tf1 = tf1_tmp.drop(labels="usershop_count", axis=1)
    usershop_feature = pd.merge(left=usershop_id, right=tf1, how="left", on=["user_id", "shop_id"])

    usershop_sample_feture = usershop_data_current_day[["instance_id", "user_id", "shop_id"]]
    usershop_feature = pd.merge(usershop_sample_feture, usershop_feature, how="left", on=["user_id","shop_id"])
    usershop_feature.drop(["user_id","shop_id"], inplace=True, axis=1)
    usershop_feature.fillna(value=0, inplace=True)
    return usershop_feature

In [ ]:
######################################################历史转化率：用户与广告品牌
def get_useritem_brand_feature(sample_data, history_data):

    #useritem_brand_id = train_data_07[["user_id", "item_brand_id"]].drop_duplicates()
    #useritem_brand_history_data = pd.merge(useritem_brand_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["user_id", "item_brand_id"])
    #useritem_brand_data = pd.merge(useritem_brand_id, train_data.loc[train_data["context_timestamp_day"] <7,:], how="inner", on=["user_id", "item_brand_id"])
    useritem_brand_data_current_day = sample_data
    useritem_brand_data = history_data 

    useritem_brand_id = useritem_brand_data.loc[:,["user_id", "item_brand_id"]].drop_duplicates()
    useritem_brand_data_is_trade = useritem_brand_data.loc[useritem_brand_data["is_trade"] == 1, :]
    useritem_brand_group = useritem_brand_data.groupby(["user_id", "item_brand_id"])
    useritem_brand_y1_group = useritem_brand_data_is_trade.groupby(["user_id", "item_brand_id"])

    tf0 = useritem_brand_group[["instance_id"]].count()
    tf0.rename(columns={"instance_id":"useritem_brand_count"},inplace=True)

    tf1_1 = tf0.copy()
    tf1_1["useritem_brand_shop_dist_count"] = useritem_brand_group["shop_id"].agg(lambda x:x.unique().size)
    tf1_2 = useritem_brand_y1_group[["instance_id"]].count()
    tf1_2.rename(columns={"instance_id": "useritem_brand_y1_shop_count"},inplace=True)
    tf1_2["useritem_brand_y1_shop_dist_count"] = useritem_brand_y1_group["user_id"].agg(lambda x:x.unique().size)
    tf1_1.reset_index(inplace=True)
    tf1_2.reset_index(inplace=True)
    tf1_tmp = pd.merge(left=tf1_1, right=tf1_2, how="left", on=["user_id", "item_brand_id"])
    tf1_tmp["useritem_brand_y1_shop_count_div_shop_count"] = tf1_tmp["useritem_brand_y1_shop_count"] / tf1_tmp["useritem_brand_count"]
    tf1_tmp["useritem_brand_y1_shop_count_div_shop_dist_count"] = tf1_tmp["useritem_brand_y1_shop_count"] / tf1_tmp["useritem_brand_shop_dist_count"]
    tf1_tmp["useritem_brand_y1_shop_dist_count_div_shop_dist_count"] = tf1_tmp["useritem_brand_y1_shop_dist_count"] / tf1_tmp["useritem_brand_shop_dist_count"]
    tf1_tmp.fillna(value=0, inplace=True)
    tf1 = tf1_tmp
    #tf1 = tf1_tmp.drop(labels="useritem_brand_count", axis=1)
    useritem_brand_feature = pd.merge(left=useritem_brand_id, right=tf1, how="left", on=["user_id", "item_brand_id"])

    useritem_brand_sample_feture = useritem_brand_data_current_day[["instance_id", "user_id", "item_brand_id"]]
    useritem_brand_feature = pd.merge(useritem_brand_sample_feture, useritem_brand_feature, how="left", on=["user_id","item_brand_id"])
    useritem_brand_feature.drop(["user_id","item_brand_id"], inplace=True, axis=1)
    useritem_brand_feature.fillna(value=0, inplace=True)
    return useritem_brand_feature

In [ ]:
def get_item_diff_feture(sample_data, history_data):
    #去重
    #sample_data = train_data_07
    item_id = sample_data.loc[:,["item_id"]].drop_duplicates()
    #history_data = pd.merge(item_id, train_data.loc[train_data["context_timestamp_day"] <7,:] ,how="inner", on="item_id")
    #去除缺失值影响
    item_sales_level_drop = history_data.loc[history_data.item_sales_level == -1,:].index
    history_data_item_sales_level = history_data.drop(item_sales_level_drop, axis=0)
    t1 = history_data_item_sales_level.groupby("item_id")[["context_timestamp"]].agg(lambda x:x.min()).reset_index() 
    t1 = pd.merge(t1, history_data[["item_sales_level", "context_timestamp", "item_id"]], how = "left", on=["item_id", "context_timestamp"])
    t1.rename(columns={"item_sales_level":"item_sales_level_history"},inplace=True)
    t1.drop_duplicates("item_id",inplace=True)
    t1 = pd.merge(sample_data[["instance_id", "item_sales_level","item_id"]], t1, how = "left", on="item_id")
    t1["item_sales_level_diff"] = t1["item_sales_level"] - t1["item_sales_level_history"]

    t2 = history_data.groupby("item_id")[["context_timestamp"]].agg(lambda x:x.min()).reset_index() 
    t2 = pd.merge(t2, history_data[["item_price_level","item_collected_level","item_pv_level", "context_timestamp", "item_id"]], how = "left", on=["item_id", "context_timestamp"])
    t2.rename(columns={"item_price_level":"item_price_level_history",
                       "item_collected_level":"item_collected_level_history",
                       "item_pv_level":"item_pv_level_history"},inplace=True)
    t2.drop_duplicates("item_id",inplace=True)
    t2 = pd.merge(sample_data[["instance_id", "item_price_level","item_collected_level","item_pv_level","item_id"]], t2, how = "left", on="item_id")
    t2["item_price_level_diff"] = t2["item_price_level"] - t2["item_price_level_history"]
    t2["item_collected_level_diff"] = t2["item_collected_level"] - t2["item_collected_level_history"]
    t2["item_pv_level_diff"] = t2["item_pv_level"] - t2["item_pv_level_history"]

    t1 = t1[["instance_id","item_sales_level_diff"]]
    t2 = t2[["instance_id","item_price_level_diff","item_collected_level_diff","item_pv_level_diff"]]
    item_diff_feature = pd.merge(t1,t2, how="left", on="instance_id")
    item_diff_feature.fillna(value=0, inplace=True)
    return item_diff_feature

In [50]:
#####################################################样本中用户行为特征提取
def get_user_sample_action_feature(input_data):
    sample_data = input_data.copy()
    user_id = sample_data.loc[:,["user_id"]].drop_duplicates()
    sample_id = sample_data[["instance_id", "user_id","shop_id", "item_id","cp_head_2", "item_brand_id"]]
    #############################################################用户当前点击的不同 广告，商店，搜索词，广告品牌的个数
    t1_group = sample_data.groupby("user_id")
    t1 = t1_group[["item_id"]].count()
    t1.rename(columns={"item_id":"user_count_today"},inplace=True)
    t1["user_item_dist_count_today"] = t1_group["item_id"].agg(lambda x:x.unique().size)
    t1["user_shop_dist_count_today"] = t1_group["shop_id"].agg(lambda x:x.unique().size)
    t1["user_cp_head_2_dist_count_today"] = t1_group["cp_head_2"].agg(lambda x:x.unique().size)
    t1["user_item_brand_dist_count_today"] = t1_group["item_brand_id"].agg(lambda x:x.unique().size)
    t1.reset_index(inplace=True)
    #t1.fillna(0, inplace=True)
    #############################################################用户当前点击特定广告的次数
    t2 = sample_data.groupby(["user_id","item_id"])[["instance_id"]].count()
    t2.rename(columns={"instance_id":"useritem_count_today"},inplace=True)
    t2.reset_index(inplace=True)               
    #############################################################用户当前点击特定商店的次数
    t3 = sample_data.groupby(["user_id","shop_id"])[["instance_id"]].count()
    t3.rename(columns={"instance_id":"usershop_count_today"},inplace=True)
    t3.reset_index(inplace=True)
    #############################################################用户当前使用特定搜索词的次数
    t4 = sample_data.groupby(["user_id","cp_head_2"])[["instance_id"]].count()
    t4.rename(columns={"instance_id":"usercp_head_2_count_today"},inplace=True)
    t4.reset_index(inplace=True)
    #############################################################用户当前点击特定广告品牌的次数
    t5 = sample_data.groupby(["user_id","item_brand_id"])[["instance_id"]].count()
    t5.rename(columns={"instance_id":"useritem_brand_count_today"},inplace=True)
    t5.reset_index(inplace=True)   
    
    t10 = sample_id[["instance_id"]]
    #sample_data["user_click_rank_today"] = sample_data.loc[:, ["user_id", "context_timestamp"]].groupby("user_id")["context_timestamp"].rank(ascending=1,method='min')
    #sample_data["usershop_click_rank_today"] = sample_data.loc[:, ["user_id", "shop_id", "context_timestamp"]].groupby(["user_id", "shop_id"])["context_timestamp"].rank(ascending=1,method='min')
    #优化sample_data["usershop_click_rank_today"] = sample_data.loc[:, ["user_id", "shop_id", "context_timestamp"]].groupby(["user_id", "shop_id"])["context_timestamp"].rank(ascending=1,method='min')
    ##############################################################用户当前是第几次点击广告
    #获取重复点击的用户
    tmp_dulicated = sample_data.duplicated(subset="user_id", keep=False)
    tmp = sample_data.loc[tmp_dulicated, :]
    #仅仅对重复点击的用户进行时间排序并编号
    tmp["user_click_rank_today"] = tmp.groupby("user_id")["context_timestamp"].rank(ascending=1,method='min')
    t11 = tmp.groupby(["user_id","context_timestamp"])["instance_id"].count().reset_index()
    t11.rename(columns={"instance_id":"user_click_same_time_count"},inplace=True)
    tmp = pd.merge(tmp, t11, how="left",on=["user_id","context_timestamp"])
    t10 = pd.merge(t10, tmp[["instance_id", "user_click_rank_today","user_click_same_time_count"]], how="left", on="instance_id")

    ############################################################用户当前在特定商店是第几次点击
    tmp_dulicated = sample_data.duplicated(subset=["user_id", "shop_id"], keep=False)
    tmp = sample_data.loc[tmp_dulicated, :]
    tmp["usershop_click_rank_today"] = tmp.groupby(["user_id", "shop_id"])["context_timestamp"].rank(ascending=1,method='min')
    #t10 = pd.merge(t10, tmp[["instance_id", "usershop_click_rank_today"]], how="left", on="instance_id")
    t11 = tmp.groupby(["user_id","shop_id","context_timestamp"])["instance_id"].count().reset_index()
    t11.rename(columns={"instance_id":"usershop_click_same_time_count"},inplace=True)
    tmp = pd.merge(tmp, t11, how="left",on=["user_id","shop_id","context_timestamp"])
    t10 = pd.merge(t10, tmp[["instance_id", "usershop_click_rank_today","usershop_click_same_time_count"]], how="left", on="instance_id")

    ############################################################用户当前在特定广告是第几次点击
    tmp_dulicated = sample_data.duplicated(subset=["user_id", "item_id"], keep=False)
    tmp = sample_data.loc[tmp_dulicated, :]
    tmp["useritem_click_rank_today"] = tmp.groupby(["user_id", "item_id"])["context_timestamp"].rank(ascending=1,method='min')
    #t10 = pd.merge(t10, tmp[["instance_id", "useritem_click_rank_today"]], how="left", on="instance_id")
    t11 = tmp.groupby(["user_id","item_id","context_timestamp"])["instance_id"].count().reset_index()
    t11.rename(columns={"instance_id":"useritem_click_same_time_count"},inplace=True)
    tmp = pd.merge(tmp, t11, how="left",on=["user_id","item_id","context_timestamp"])
    t10 = pd.merge(t10, tmp[["instance_id", "useritem_click_rank_today","useritem_click_same_time_count"]], how="left", on="instance_id")

    ############################################################用户当前在特定搜索是第几次点击
    tmp_dulicated = sample_data.duplicated(subset=["user_id", "cp_head_2"], keep=False)
    tmp = sample_data.loc[tmp_dulicated, :]
    tmp["usercp_head_2_click_rank_today"] = tmp.groupby(["user_id", "cp_head_2"])["context_timestamp"].rank(ascending=1,method='min')
    #t10 = pd.merge(t10, tmp[["instance_id", "usercp_head_2_click_rank_today"]], how="left", on="instance_id")
    t11 = tmp.groupby(["user_id","cp_head_2","context_timestamp"])["instance_id"].count().reset_index()
    t11.rename(columns={"instance_id":"usercp_head_2_click_same_time_count"},inplace=True)
    tmp = pd.merge(tmp, t11, how="left",on=["user_id","cp_head_2","context_timestamp"])
    t10 = pd.merge(t10, tmp[["instance_id", "usercp_head_2_click_rank_today","usercp_head_2_click_same_time_count"]], how="left", on="instance_id")
    
    tmp_dulicated = sample_data.duplicated(subset=["user_id", "item_brand_id"], keep=False)
    tmp = sample_data.loc[tmp_dulicated, :]
    tmp["useritem_brand_click_rank_today"] = tmp.groupby(["user_id", "item_brand_id"])["context_timestamp"].rank(ascending=1,method='min')
    #t10 = pd.merge(t10, tmp[["instance_id", "useritem_brand_click_rank_today"]], how="left", on="instance_id")
    t11 = tmp.groupby(["user_id","item_brand_id","context_timestamp"])["instance_id"].count().reset_index()
    t11.rename(columns={"instance_id":"useritem_brand_click_same_time_count"},inplace=True)
    tmp = pd.merge(tmp, t11, how="left",on=["user_id","item_brand_id","context_timestamp"])
    t10 = pd.merge(t10, tmp[["instance_id", "useritem_brand_click_rank_today","useritem_brand_click_same_time_count"]], how="left", on="instance_id")

    
    t10.fillna(value=1.0,inplace=True)
    ############################################################用户当前点击的商店是第几个不同的商店
    #按时间对样本进行排序，删除重复的user_id和shop_id

    tmp_1 = sample_data.copy()
    tmp_1.sort_values(by=["context_timestamp", "instance_id"],ascending=[1,1],inplace=True)

    tmp = tmp_1.copy()
    tmp.drop_duplicates(["user_id", "shop_id"], keep="first",inplace=True)
    #根据时间对user_id进行组内排序并标号，然后与原始的sample_data合并
    tmp["user_dist_shop_click_rank_today"] = tmp.groupby("user_id")["context_timestamp"].rank(ascending=1,method='min')
    tmp = pd.merge(tmp_1, tmp[["instance_id", "user_dist_shop_click_rank_today"]], how="left", on="instance_id")
    #对user_id和时间进行增序排序，模拟groupby user_id rank timestamp
    tmp.sort_values(by=["user_id","context_timestamp","instance_id"],ascending=[1,1,1],inplace=True)
    #生成自然序列索引并放到列中
    tmp.reset_index(inplace=True,drop=True)
#    tmp.reset_index(inplace=True)
    #将合并后的空值是user_id中的重复shop_id,将其序号赋值为上一个索引的值
#     def fill_same_shop_rank_with_previous(x):
#         if np.isnan(x["user_dist_shop_click_rank_today"]) == True:
#             #print(x["index"])
#             previous = tmp.loc[x["index"] - 1, "user_dist_shop_click_rank_today"]
#             tmp.loc[x["index"], "user_dist_shop_click_rank_today"] = previous
#             return previous
#         else:
#             return x.user_dist_shop_click_rank_today
#     tmp["user_dist_shop_click_rank_today"] = tmp.apply(fill_same_shop_rank_with_previous,axis=1)
    tmp_2 = tmp.loc[tmp["user_dist_shop_click_rank_today"].isnull()!=True, ["user_dist_shop_click_rank_today"]]
    tmp["user_dist_shop_click_rank_today"] = tmp_2["user_dist_shop_click_rank_today"].reindex(index=range(tmp.shape[0]), method="ffill")
    t10 = pd.merge(t10, tmp[["instance_id", "user_dist_shop_click_rank_today"]], how="left", on="instance_id")

    ############################################################用户当前点击的广告是第几个不同的广告
    #按时间对样本进行排序，删除重复的user_id和item_id
    tmp = tmp_1.copy()
    tmp.drop_duplicates(["user_id", "item_id"], keep="first",inplace=True)
    #根据时间对user_id进行组内排序并标号，然后与原始的sample_data合并
    tmp["user_dist_item_click_rank_today"] = tmp.groupby("user_id")["context_timestamp"].rank(ascending=1,method='min')
    tmp = pd.merge(tmp_1, tmp[["instance_id", "user_dist_item_click_rank_today"]], how="left", on="instance_id")
    #对user_id和时间进行增序排序，模拟groupby user_id rank timestamp
    tmp.sort_values(by=["user_id","context_timestamp","instance_id"],ascending=[1,1,1],inplace=True)
    #生成自然序列索引并放到列中
    tmp.reset_index(inplace=True,drop=True)
    #tmp.reset_index(inplace=True)
    #合并后的空值是user_id中的重复item_id,将其序号赋值为上一个索引的值,(同时将原始数据中的当前数据也赋值成上一个索引值，否则连续两个nan会出错)
#     def fill_same_item_rank_with_previous(x):
#         if np.isnan(x["user_dist_item_click_rank_today"]) == True:
#             #print(x["index"])
#             previous = tmp.loc[x["index"] - 1, "user_dist_item_click_rank_today"]
#             tmp.loc[x["index"], "user_dist_item_click_rank_today"] = previous
#             return previous
#         else:
#             return x.user_dist_item_click_rank_today
#     tmp["user_dist_item_click_rank_today"] = tmp.apply(fill_same_item_rank_with_previous,axis=1)
    tmp_2 = tmp.loc[tmp["user_dist_item_click_rank_today"].isnull()!=True, ["user_dist_item_click_rank_today"]]
    tmp["user_dist_item_click_rank_today"] = tmp_2["user_dist_item_click_rank_today"].reindex(index=range(tmp.shape[0]), method="ffill")
    t10 = pd.merge(t10, tmp[["instance_id", "user_dist_item_click_rank_today"]], how="left", on="instance_id")

    t_tmp = pd.merge(sample_id, t1, how="left", on="user_id")
    t_tmp = pd.merge(t_tmp, t2, how="left", on=["user_id", "item_id"])
    t_tmp = pd.merge(t_tmp, t3, how="left", on=["user_id", "shop_id"])
    t_tmp = pd.merge(t_tmp, t4, how="left", on=["user_id", "cp_head_2"])
    t_tmp = pd.merge(t_tmp, t5, how="left", on=["user_id", "item_brand_id"])
    t_tmp = pd.merge(t_tmp, t10, how="left", on="instance_id")
    t_tmp["user_click_rank_today"] = t_tmp["user_click_rank_today"] - 1
    t_tmp["user_click_next_today_count"] = t_tmp["user_count_today"] - t_tmp["user_click_same_time_count"] - t_tmp["user_click_rank_today"]
    t_tmp["usershop_click_rank_today"] = t_tmp["usershop_click_rank_today"] - 1
    t_tmp["usershop_click_next_today_count"] = t_tmp["usershop_count_today"] - t_tmp["usershop_click_same_time_count"] - t_tmp["usershop_click_rank_today"]
    t_tmp["useritem_click_rank_today"] = t_tmp["useritem_click_rank_today"] - 1
    t_tmp["useritem_click_next_today_count"] = t_tmp["useritem_count_today"] - t_tmp["useritem_click_same_time_count"] - t_tmp["useritem_click_rank_today"]
    t_tmp["usercp_head_2_click_rank_today"] = t_tmp["usercp_head_2_click_rank_today"] - 1
    t_tmp["usercp_head_2_click_next_today_count"] = t_tmp["usercp_head_2_count_today"] - t_tmp["usercp_head_2_click_same_time_count"] - t_tmp["usercp_head_2_click_rank_today"]
    t_tmp["useritem_brand_click_rank_today"] = t_tmp["useritem_brand_click_rank_today"] - 1
    t_tmp["useritem_brand_click_next_today_count"] = t_tmp["useritem_brand_count_today"] - t_tmp["useritem_brand_click_same_time_count"] - t_tmp["useritem_brand_click_rank_today"]

    t_tmp.drop(["user_id","shop_id", "item_id","cp_head_2", "item_brand_id"], inplace=True, axis=1)
    sample_action_feature = t_tmp
    return sample_action_feature


In [51]:
#####################################################样本中用户行为特征提取(用户多次点击时间间隔)
def get_user_action_v2_feature(sample_data, history_data):
    #去重
    user_id = sample_data.loc[:,["user_id"]].drop_duplicates()
    half_history_data = pd.merge(user_id, history_data,how="inner", on="user_id")

    sample_id = sample_data[["instance_id", "user_id","shop_id", "item_id","cp_head_2","item_brand_id"]]
    #tmp_dulicated = sample_data.duplicated(subset="user_id", keep=False)
    tmp_data = pd.concat([sample_data, half_history_data])
    #仅仅对重复点击的用户进行时间排序并编号
    #######################################用户上一次/下一次点击的时间间隔  
    tmp_dulicated = tmp_data.duplicated(subset="user_id", keep=False)
    tmp_dulicated_data = tmp_data.loc[tmp_dulicated, :]
    tmp_drop_usertime = tmp_dulicated_data.drop_duplicates(subset=["user_id", "context_timestamp"])
    tmp_drop_usertime.sort_values(["user_id", "context_timestamp"], ascending=[1,1], inplace=True)
    tmp_drop_usertime["user_last_click_time_interval"] = tmp_drop_usertime.groupby("user_id")["context_timestamp"].diff(periods=1)
    tmp_drop_usertime["user_next_click_time_interval"] = -tmp_drop_usertime.groupby("user_id")["context_timestamp"].diff(periods=-1)
    tmp_dulicated_data = pd.merge(tmp_dulicated_data, tmp_drop_usertime[["user_last_click_time_interval", "user_next_click_time_interval","user_id", "context_timestamp"]], how="left", on=["user_id", "context_timestamp"])
    action_v2_feature = pd.merge(sample_id[["instance_id"]], tmp_dulicated_data[["instance_id", "user_last_click_time_interval", "user_next_click_time_interval"]], how="left", on="instance_id")

    #######################################用户上一次/下一次点击特定商店的时间间隔    
    usershop_id = sample_data.loc[:,["user_id","shop_id"]].drop_duplicates()
    half_history_data = pd.merge(usershop_id, history_data,how="inner", on=["user_id", "shop_id"])

    #sample_id = sample_data[["instance_id", "user_id","shop_id", "item_id","cp_head_2"]]
    
    tmp_data = pd.concat([sample_data, half_history_data])
    tmp_dulicated = tmp_data.duplicated(subset=["user_id","shop_id"], keep=False)
    tmp_dulicated_data = tmp_data.loc[tmp_dulicated, :]
    #仅仅对重复点击的用户进行时间排序并编号
    tmp_dulicated_data.sort_values(["user_id","shop_id", "context_timestamp", "instance_id"], ascending=[1,1,1,1], inplace=True)
    tmp_dulicated_data["usershop_last_click_time_interval"] = tmp_dulicated_data.groupby(["user_id","shop_id"])["context_timestamp"].diff(periods=1)
    tmp_dulicated_data["usershop_next_click_time_interval"] = -tmp_dulicated_data.groupby(["user_id","shop_id"])["context_timestamp"].diff(periods=-1)
    action_v2_feature = pd.merge(action_v2_feature, tmp_dulicated_data[["instance_id", "usershop_last_click_time_interval", "usershop_next_click_time_interval"]], how="left", on="instance_id")

    #######################################用户上一次/下一次点击特定广告的时间间隔
    useritem_id = sample_data.loc[:,["user_id","item_id"]].drop_duplicates()
    half_history_data = pd.merge(useritem_id, history_data,how="inner", on=["user_id", "item_id"])

    tmp_data = pd.concat([sample_data, half_history_data])
    tmp_dulicated = tmp_data.duplicated(subset=["user_id","item_id"], keep=False)
    tmp_dulicated_data = tmp_data.loc[tmp_dulicated, :]

    tmp_dulicated_data.sort_values(["user_id","item_id", "context_timestamp", "instance_id"], ascending=[1,1,1,1], inplace=True)
    tmp_dulicated_data["useritem_last_click_time_interval"] = tmp_dulicated_data.groupby(["user_id","item_id"])["context_timestamp"].diff(periods=1)
    tmp_dulicated_data["useritem_next_click_time_interval"] = -tmp_dulicated_data.groupby(["user_id","item_id"])["context_timestamp"].diff(periods=-1)
    action_v2_feature = pd.merge(action_v2_feature, tmp_dulicated_data[["instance_id", "useritem_last_click_time_interval", "useritem_next_click_time_interval"]], how="left", on="instance_id")
    
    #######################################用户上一次/下一次点击特定搜索的时间间隔
    usercp_head_2 = sample_data.loc[:,["user_id","cp_head_2"]].drop_duplicates()
    half_history_data = pd.merge(usercp_head_2, history_data,how="inner", on=["user_id", "cp_head_2"])

    tmp_data = pd.concat([sample_data, half_history_data])
    tmp_dulicated = tmp_data.duplicated(subset=["user_id","cp_head_2"], keep=False)
    tmp_dulicated_data = tmp_data.loc[tmp_dulicated, :]

    tmp_dulicated_data.sort_values(["user_id","cp_head_2", "context_timestamp", "instance_id"], ascending=[1,1,1,1], inplace=True)
    tmp_dulicated_data["usercp_head_2_last_click_time_interval"] = tmp_dulicated_data.groupby(["user_id","cp_head_2"])["context_timestamp"].diff(periods=1)
    tmp_dulicated_data["usercp_head_2_next_click_time_interval"] = -tmp_dulicated_data.groupby(["user_id","cp_head_2"])["context_timestamp"].diff(periods=-1)
    action_v2_feature = pd.merge(action_v2_feature, tmp_dulicated_data[["instance_id", "usercp_head_2_last_click_time_interval", "usercp_head_2_next_click_time_interval"]], how="left", on="instance_id")

    #######################################用户上一次/下一次点击特定广告品牌的时间间隔
    useritem_brand_id = sample_data.loc[:,["user_id","item_brand_id"]].drop_duplicates()
    half_history_data = pd.merge(useritem_brand_id, history_data,how="inner", on=["user_id", "item_brand_id"])

    tmp_data = pd.concat([sample_data, half_history_data])
    tmp_dulicated = tmp_data.duplicated(subset=["user_id","item_brand_id"], keep=False)
    tmp_dulicated_data = tmp_data.loc[tmp_dulicated, :]

    tmp_dulicated_data.sort_values(["user_id","item_brand_id", "context_timestamp", "instance_id"], ascending=[1,1,1,1], inplace=True)
    tmp_dulicated_data["useritem_brand_last_click_time_interval"] = tmp_dulicated_data.groupby(["user_id","item_brand_id"])["context_timestamp"].diff(periods=1)
    tmp_dulicated_data["useritem_brand_next_click_time_interval"] = -tmp_dulicated_data.groupby(["user_id","item_brand_id"])["context_timestamp"].diff(periods=-1)
    action_v2_feature = pd.merge(action_v2_feature, tmp_dulicated_data[["instance_id", "useritem_brand_last_click_time_interval", "useritem_brand_next_click_time_interval"]], how="left", on="instance_id")
    
    
    #time_half_day = 43200
    time_half_day = 86400
    action_v2_feature.fillna(value=time_half_day,inplace=True)
    action_v2_feature.loc[action_v2_feature["user_last_click_time_interval"]>time_half_day, "user_last_click_time_interval"] = time_half_day
    action_v2_feature.loc[action_v2_feature["user_next_click_time_interval"]>time_half_day, "user_next_click_time_interval"] = time_half_day
    action_v2_feature.loc[action_v2_feature["usershop_last_click_time_interval"]>time_half_day, "usershop_last_click_time_interval"] = time_half_day
    action_v2_feature.loc[action_v2_feature["usershop_next_click_time_interval"]>time_half_day, "usershop_next_click_time_interval"] = time_half_day
    action_v2_feature.loc[action_v2_feature["useritem_last_click_time_interval"]>time_half_day, "useritem_last_click_time_interval"] = time_half_day
    action_v2_feature.loc[action_v2_feature["useritem_next_click_time_interval"]>time_half_day, "useritem_next_click_time_interval"] = time_half_day
    action_v2_feature.loc[action_v2_feature["usercp_head_2_last_click_time_interval"]>time_half_day, "usercp_head_2_last_click_time_interval"] = time_half_day
    action_v2_feature.loc[action_v2_feature["usercp_head_2_next_click_time_interval"]>time_half_day, "usercp_head_2_next_click_time_interval"] = time_half_day
    action_v2_feature.loc[action_v2_feature["useritem_brand_last_click_time_interval"]>time_half_day, "useritem_brand_last_click_time_interval"] = time_half_day
    action_v2_feature.loc[action_v2_feature["useritem_brand_next_click_time_interval"]>time_half_day, "useritem_brand_next_click_time_interval"] = time_half_day

    return action_v2_feature

In [60]:
def get_user_action_v3_feature(sample_data, history_data):
    time_1_hours = 3600
    user_id = sample_data.loc[:,["user_id"]].drop_duplicates()
    #获取需要的历史用户
    history_data_used = pd.merge(user_id, history_data, how="inner", on="user_id")

    sample_id = sample_data[["instance_id", "user_id","shop_id", "item_id","cp_head_2"]]
    #合并当前样本和历史数据
    tmp_data = pd.concat([sample_data, history_data_used])
    #获取重复出现的用户
    tmp_dulicated = tmp_data.duplicated(subset="user_id", keep=False)
    tmp_dulicated_data = tmp_data.loc[tmp_dulicated, :]
    ####################################################用户在当前样本时间之前的两小时内点击次数
    tmp_dulicated_data.sort_values(["user_id", "context_timestamp", "instance_id"], ascending=[1,1,1], inplace=True)
    def get_time_before_count_in_group(x):
        global tmp_group
        c = len(np.where((tmp_group.context_timestamp >(x-time_1_hours)) & (tmp_group.context_timestamp <x))[0])
        return c
    def get_time_before_count(x):
        #print(x)
        global tmp_group
        tmp_group = x[["context_timestamp"]]
        #在tmp_group上再创建一列消耗约5秒的时间，因此在外面reset_index即可
        #tmp_group["user_click_count_in_last_2_hours"] = tmp_group.context_timestamp.apply(get_time_before_count)
        #return tmp_group[["user_click_count_in_last_2_hours"]]
        return tmp_group.context_timestamp.apply(get_time_before_count_in_group)

    tmp_out = tmp_dulicated_data.groupby("user_id").apply(get_time_before_count)
    tmp_out = tmp_out.reset_index()
    tmp_out.rename(columns={"context_timestamp":"user_click_count_in_last_2_hours"}, inplace=True)
    tmp_out.set_index("level_1",inplace=True)
    tmp_dulicated_data["user_click_count_in_last_2_hours"] = tmp_out["user_click_count_in_last_2_hours"]
    action_v3_feature = pd.merge(sample_id[["instance_id"]], tmp_dulicated_data[["instance_id", "user_click_count_in_last_2_hours"]], how="left", on="instance_id")
    print("last ok")
    ####################################################用户在当前样本时间之后的两小时内点击次数
    def get_time_later_count_in_group(x):
        global tmp_group
        c = len(np.where((tmp_group.context_timestamp <(x+time_1_hours)) & (tmp_group.context_timestamp >x))[0])
        return c
    def get_time_later_count(x):
        #print(x)
        global tmp_group
        tmp_group = x[["context_timestamp"]]
        #在tmp_group上再创建一列消耗约5秒的时间，因此在外面reset_index即可
        #tmp_group["user_click_count_in_last_2_hours"] = tmp_group.context_timestamp.apply(get_time_before_count)
        #return tmp_group[["user_click_count_in_last_2_hours"]]
        return tmp_group.context_timestamp.apply(get_time_later_count_in_group)

    tmp_dulicated = sample_data.duplicated(subset="user_id", keep=False)
    tmp_dulicated_data = sample_data.loc[tmp_dulicated, :]
    tmp_dulicated_data.sort_values(["user_id", "context_timestamp", "instance_id"], ascending=[1,1,1], inplace=True)

    tmp_out = tmp_dulicated_data.groupby("user_id").apply(get_time_later_count)
    tmp_out = tmp_out.reset_index()
    tmp_out.rename(columns={"context_timestamp":"user_click_count_in_later_2_hours"}, inplace=True)
    tmp_out.set_index("level_1",inplace=True)
    tmp_dulicated_data["user_click_count_in_later_2_hours"] = tmp_out["user_click_count_in_later_2_hours"]
    print("later ok")


    action_v3_feature = pd.merge(action_v3_feature, tmp_dulicated_data[["instance_id","user_click_count_in_later_2_hours"]], how="left", on="instance_id")
    action_v3_feature.fillna(value=0,inplace=True)
    return action_v3_feature

In [198]:
############################################唯一个数和缺失个数
def get_summary_data(data, miss_symbol):
    input_data = data.replace(miss_symbol, np.NaN)
    distinct_count = input_data.apply(lambda x: x.unique().size)
    dismiss_count = input_data.apply(lambda x: x.count())
    distinct_count.rename("distinct_count", inplace=True)
    dismiss_count.rename("dismiss_count", inplace=True)
    summary_data = pd.concat([distinct_count, dismiss_count],axis=1)
    summary_data.insert(loc=0, column="row_index",value=range(summary_data.index.size))
    summary_data["miss_count"] = input_data.index.size - summary_data.dismiss_count
    summary_data["variable_type"] = input_data.dtypes
    summary_data["min_value"] = input_data.min()
    summary_data["max_value"] = input_data.max()
    #summary_data["row_index"] = range(summary_data.index.size)
    #不能先重建索引，否则后面就无法根据索引新增数据了
    #summary_data.reset_index(inplace=True)
    #summary_data.rename(columns={"index":"column_name"}, inplace=True)
    return summary_data